# 데이터 전처리

In [ ]:
import os
import sys
import pandas as pd
import polars as pl

# 상위 폴더의 src를 인식하기 위한 경로 설정
sys.path.append(os.path.abspath(os.path.join('..')))

from src.data_loader import merge_csv_data, preprocess_population

## 점포정보 & 추정매출 데이터 전처리

In [ ]:
# -*- coding: utf-8 -*-
# 점포정보 & 추정매출 처리 (이미 분기 데이터)

import pandas as pd
import glob
import os
import sys
import warnings
warnings.filterwarnings('ignore')

os.makedirs("output", exist_ok=True)

# ====== 1. 점포정보 처리 ======
print("="*60)
print("📂 점포정보 데이터 처리")
print("="*60)

store_files = sorted(glob.glob("data/store/*.csv"))
print(f"발견된 파일: {len(store_files)}개\n")

store_list = []

for i, file in enumerate(store_files, 1):
    print(f"[{i}/{len(store_files)}] {os.path.basename(file)}")
    
    # 인코딩 시도
    for enc in ['utf-8', 'cp949', 'euc-kr']:
        try:
            df = pd.read_csv(file, encoding=enc)
            print(f"  ✓ 읽기 성공: {df.shape}")
            store_list.append(df)
            break
        except:
            continue

# 점포정보 통합
if store_list:
    df_store = pd.concat(store_list, ignore_index=True)
    print(f"\n점포정보 통합: {df_store.shape}")
    
    # 컬럼명 확인
    print(f"컬럼: {df_store.columns.tolist()[:5]}...")
    
    # 저장
    df_store.to_csv("output/분기집계_점포정보.csv", index=False, encoding='utf-8-sig')
    print(f"✅ 저장 완료: output/분기집계_점포정보.csv")

# ====== 2. 추정매출 처리 ======
print("\n" + "="*60)
print("📂 추정매출 데이터 처리")
print("="*60)

sales_files = sorted(glob.glob("data/sales/*.csv"))
print(f"발견된 파일: {len(sales_files)}개\n")

sales_list = []

for i, file in enumerate(sales_files, 1):
    print(f"[{i}/{len(sales_files)}] {os.path.basename(file)}")
    
    # 인코딩 시도
    for enc in ['utf-8', 'cp949', 'euc-kr']:
        try:
            df = pd.read_csv(file, encoding=enc)
            print(f"  ✓ 읽기 성공: {df.shape}")
            sales_list.append(df)
            break
        except:
            continue

# 추정매출 통합
if sales_list:
    df_sales = pd.concat(sales_list, ignore_index=True)
    print(f"\n추정매출 통합: {df_sales.shape}")
    
    # 컬럼명 확인
    print(f"컬럼: {df_sales.columns.tolist()[:5]}...")
    
    # 저장
    df_sales.to_csv("output/분기집계_추정매출.csv", index=False, encoding='utf-8-sig')
    print(f"✅ 저장 완료: output/분기집계_추정매출.csv")

print("\n" + "="*60)
print("✅ 모든 처리 완료!")
print("="*60)

## 생활인구 데이터 전처리

step0) 폴더 준비

In [43]:
from pathlib import Path
BASE = Path(r"C:\Users\user\OneDrive\dataset")
SRC_POP = BASE / r"data\population"
OUT     = BASE / r"output\pop_q_parquet"
OUT.mkdir(parents=True, exist_ok=True)
print("원본:", SRC_POP)
print("산출:", OUT)


원본: C:\Users\user\OneDrive\dataset\data\population
산출: C:\Users\user\OneDrive\dataset\output\pop_q_parquet


step1) CSV → 월별 SLOT Parquet 파일로 저장

In [53]:
import re
import polars as pl
from pathlib import Path

def sniff_delim(path: Path, kbytes: int = 64) -> str:
    cand = [",", ";", "|", "\t"]
    with open(path, "rb") as f:
        head = f.read(kbytes * 1024)
    text = head.decode("utf-8", errors="ignore")
    counts = {d: text.count(d) for d in cand}
    return max(counts, key=counts.get) or ","

def csv_to_parquet_monthwise(src_csv: Path, yyyymm: str, out_dir: Path):
    dst = out_dir / f"POP_Q_SLOT_{yyyymm}.parquet"
    sep = sniff_delim(src_csv)

    # 시도 1) scan_csv (가장 빠름)
    try:
        lf = pl.scan_csv(
            str(src_csv),
            has_header=True,
            separator=sep,
            encoding="utf8-lossy",
            infer_schema_length=100_000,
            ignore_errors=True,
            low_memory=True,
            quote_char='"',              # 따옴표 없으면 아래 폴백에서 None로 재시도됨
            null_values=["", "NA", "N/A"],
            truncate_ragged_lines=True   # 줄마다 필드 수 불일치 허용
        )
        # ID/코드 류를 문자열로 고정
        id_like = [c for c in lf.columns if any(k in c for k in ["코드","code","ID","id","Code"])]
        if id_like:
            lf = lf.with_columns([pl.col(c).cast(pl.Utf8) for c in id_like])

        lf.sink_parquet(str(dst))
        print(f"✅ SLOT 저장: {dst.name} (sep='{sep}', mode=scan_csv)")
        return dst

    except TypeError:
        # 예: quote_char 타입 문제 등
        pass
    except Exception:
        pass

    # 시도 2) scan_csv 재시도(quote_char 없음)
    try:
        lf = pl.scan_csv(
            str(src_csv),
            has_header=True,
            separator=sep,
            encoding="utf8-lossy",
            infer_schema_length=100_000,
            ignore_errors=True,
            low_memory=True,
            quote_char=None,
            null_values=["", "NA", "N/A"],
            truncate_ragged_lines=True
        )
        id_like = [c for c in lf.columns if any(k in c for k in ["코드","code","ID","id","Code"])]
        if id_like:
            lf = lf.with_columns([pl.col(c).cast(pl.Utf8) for c in id_like])

        lf.sink_parquet(str(dst))
        print(f"✅ SLOT 저장: {dst.name} (sep='{sep}', mode=scan_csv, quote=None)")
        return dst

    except Exception:
        pass

    # 시도 3) read_csv(메모리 사용↑)로 폴백
    df = pl.read_csv(
        str(src_csv),
        has_header=True,
        separator=sep,
        encoding="utf8-lossy",
        infer_schema_length=100_000,
        ignore_errors=True,
        quote_char=None,
        null_values=["", "NA", "N/A"],
        truncate_ragged_lines=True
    )
    id_like = [c for c in df.columns if any(k in c for k in ["코드","code","ID","id","Code"])]
    if id_like:
        df = df.with_columns([pl.col(c).cast(pl.Utf8) for c in id_like])

    df.write_parquet(str(dst))
    print(f"✅ SLOT 저장: {dst.name} (sep='{sep}', mode=read_csv fallback)")
    return dst

# 실행
for yyyymm, f in iter_month_csvs(SRC_POP):
    csv_to_parquet_monthwise(f, yyyymm, OUT)


C:\Users\user\AppData\Local\Temp\ipykernel_6640\341172022.py:32: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  id_like = [c for c in lf.columns if any(k in c for k in ["코드","code","ID","id","Code"])]


✅ SLOT 저장: POP_Q_SLOT_201901.parquet (sep=',', mode=scan_csv)
✅ SLOT 저장: POP_Q_SLOT_201901.parquet (sep=',', mode=scan_csv)
✅ SLOT 저장: POP_Q_SLOT_201902.parquet (sep=',', mode=scan_csv)
✅ SLOT 저장: POP_Q_SLOT_201903.parquet (sep=',', mode=scan_csv)
✅ SLOT 저장: POP_Q_SLOT_201904.parquet (sep=',', mode=scan_csv)
✅ SLOT 저장: POP_Q_SLOT_201905.parquet (sep=',', mode=scan_csv)
✅ SLOT 저장: POP_Q_SLOT_201906.parquet (sep=',', mode=scan_csv)
✅ SLOT 저장: POP_Q_SLOT_201907.parquet (sep=',', mode=scan_csv)
✅ SLOT 저장: POP_Q_SLOT_201908.parquet (sep=',', mode=scan_csv)
✅ SLOT 저장: POP_Q_SLOT_201909.parquet (sep=',', mode=scan_csv)
✅ SLOT 저장: POP_Q_SLOT_201910.parquet (sep=',', mode=scan_csv)
✅ SLOT 저장: POP_Q_SLOT_201911.parquet (sep=',', mode=scan_csv)
✅ SLOT 저장: POP_Q_SLOT_201912.parquet (sep=',', mode=scan_csv)
✅ SLOT 저장: POP_Q_SLOT_202001.parquet (sep=',', mode=scan_csv)
✅ SLOT 저장: POP_Q_SLOT_202002.parquet (sep=',', mode=scan_csv)
✅ SLOT 저장: POP_Q_SLOT_202003.parquet (sep=',', mode=scan_csv)
✅ SLOT 저

step2) 월별 NORM 생성 - 10세구간합 & 시간대6개구분

In [105]:
import polars as pl, re
from pathlib import Path

OUT = Path(r"C:\Users\user\OneDrive\dataset\output\pop_q_parquet")

def hour_to_slot6_expr(hour_col: str) -> pl.Expr:
    return (
        pl.when(pl.col(hour_col).is_between(0, 3)).then(pl.lit("00~06"))
        .when(pl.col(hour_col).is_between(4, 7)).then(pl.lit("06~11"))
        .when(pl.col(hour_col).is_between(8, 11)).then(pl.lit("11~14"))
        .when(pl.col(hour_col).is_between(12, 15)).then(pl.lit("14~17"))
        .when(pl.col(hour_col).is_between(16, 19)).then(pl.lit("17~21"))
        .otherwise(pl.lit("21~24"))
    )

def yyyymm_to_parts(yyyymm: str):
    y = int(yyyymm[:4]); m = int(yyyymm[4:])
    q = (m-1)//3 + 1
    return y, m, q, int(f"{y}{q}")

def build_age10_manual(df):
    return df.with_columns([
        (pl.col("여자0세부터9세생활인구수")).alias("여자10대미만_생활인구수"),
        (pl.col("여자10세부터14세생활인구수") + pl.col("여자15세부터19세생활인구수")).alias("여자10대_생활인구수"),
        (pl.col("여자20세부터24세생활인구수") + pl.col("여자25세부터29세생활인구수")).alias("여자20대_생활인구수"),
        (pl.col("여자30세부터34세생활인구수") + pl.col("여자35세부터39세생활인구수")).alias("여자30대_생활인구수"),
        (pl.col("여자40세부터44세생활인구수") + pl.col("여자45세부터49세생활인구수")).alias("여자40대_생활인구수"),
        (pl.col("여자50세부터54세생활인구수") + pl.col("여자55세부터59세생활인구수")).alias("여자50대_생활인구수"),
        (pl.col("여자60세부터64세생활인구수") + pl.col("여자65세부터69세생활인구수")).alias("여자60대_생활인구수"),
        pl.col("여자70세이상생활인구수").alias("여자70대이상_생활인구수"),

        (pl.col("남자0세부터9세생활인구수")).alias("남자10대미만_생활인구수"),
        (pl.col("남자10세부터14세생활인구수") + pl.col("남자15세부터19세생활인구수")).alias("남자10대_생활인구수"),
        (pl.col("남자20세부터24세생활인구수") + pl.col("남자25세부터29세생활인구수")).alias("남자20대_생활인구수"),
        (pl.col("남자30세부터34세생활인구수") + pl.col("남자35세부터39세생활인구수")).alias("남자30대_생활인구수"),
        (pl.col("남자40세부터44세생활인구수") + pl.col("남자45세부터49세생활인구수")).alias("남자40대_생활인구수"),
        (pl.col("남자50세부터54세생활인구수") + pl.col("남자55세부터59세생활인구수")).alias("남자50대_생활인구수"),
        (pl.col("남자60세부터64세생활인구수") + pl.col("남자65세부터69세생활인구수")).alias("남자60대_생활인구수"),
        pl.col("남자70세이상생활인구수").alias("남자70대이상_생활인구수"),
    ])

def sales_label_expr(slot_col: str) -> pl.Expr:
    # "시간대_00~06_생활인구수" 같은 최종 컬럼 라벨
    return pl.lit("시간대_") + pl.col(slot_col) + pl.lit("_생활인구수")

HOUR_CANDIDATES = ["시간대구분"]
KEYS_KEEP = ["행정동코드"]

for p in sorted(OUT.glob("POP_Q_SLOT_??????.parquet")):
    yyyymm = re.search(r"(\d{6})", p.name).group(1)
    y, m, q, yq = yyyymm_to_parts(yyyymm)
    dst = p.with_name(f"POP_Q_SLOT_NORM_{yyyymm}.parquet")

    lf = pl.scan_parquet(str(p))

    # 분기/월 코드
    lf = lf.with_columns([
        pl.lit(y).alias("기준년도"),
        pl.lit(q).alias("기준분기"),
        pl.lit(yq).alias("기준년분기_코드"),
        pl.lit(int(yyyymm)).alias("기준년월코드"),
    ])

    # 시간대 컬럼이 숫자인지 확인
    lf = lf.with_columns(pl.col("시간대구분").cast(pl.Int32))
    lf = lf.with_columns(hour_to_slot6_expr("시간대구분").alias("시간대라벨"))

    # 10세 구간 생성
    lf = build_age10_manual(lf)

    df_cols_after = lf.columns
    keep = [c for c in KEYS_KEEP if c in df_cols_after] + [
        "기준년도","기준분기","기준년분기_코드","기준년월코드","시간대라벨","총생활인구수"
    ] + [c for c in df_cols_after if c.endswith("_생활인구수") and (c.startswith("여자") or c.startswith("남자"))]

    lf.select(keep).collect().write_parquet(str(dst))
    print(f"✅ NORM 생성: {dst.name}")


C:\Users\user\AppData\Local\Temp\ipykernel_6640\1241723409.py:71: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  df_cols_after = lf.columns


✅ NORM 생성: POP_Q_SLOT_NORM_201901.parquet
✅ NORM 생성: POP_Q_SLOT_NORM_201902.parquet
✅ NORM 생성: POP_Q_SLOT_NORM_201903.parquet
✅ NORM 생성: POP_Q_SLOT_NORM_201904.parquet
✅ NORM 생성: POP_Q_SLOT_NORM_201905.parquet
✅ NORM 생성: POP_Q_SLOT_NORM_201906.parquet
✅ NORM 생성: POP_Q_SLOT_NORM_201907.parquet
✅ NORM 생성: POP_Q_SLOT_NORM_201908.parquet
✅ NORM 생성: POP_Q_SLOT_NORM_201909.parquet
✅ NORM 생성: POP_Q_SLOT_NORM_201910.parquet
✅ NORM 생성: POP_Q_SLOT_NORM_201911.parquet
✅ NORM 생성: POP_Q_SLOT_NORM_201912.parquet
✅ NORM 생성: POP_Q_SLOT_NORM_202001.parquet
✅ NORM 생성: POP_Q_SLOT_NORM_202002.parquet
✅ NORM 생성: POP_Q_SLOT_NORM_202003.parquet
✅ NORM 생성: POP_Q_SLOT_NORM_202004.parquet
✅ NORM 생성: POP_Q_SLOT_NORM_202005.parquet
✅ NORM 생성: POP_Q_SLOT_NORM_202006.parquet
✅ NORM 생성: POP_Q_SLOT_NORM_202007.parquet
✅ NORM 생성: POP_Q_SLOT_NORM_202008.parquet
✅ NORM 생성: POP_Q_SLOT_NORM_202009.parquet
✅ NORM 생성: POP_Q_SLOT_NORM_202010.parquet
✅ NORM 생성: POP_Q_SLOT_NORM_202011.parquet
✅ NORM 생성: POP_Q_SLOT_NORM_202012.

★점검

In [103]:
import polars as pl, random
from pathlib import Path

OUT = Path(r"C:\Users\user\OneDrive\dataset\output\pop_q_parquet")

files = sorted(OUT.glob("POP_Q_SLOT_NORM_*.parquet"))
for f in random.sample(files, k=min(6, len(files))):
    df = pl.read_parquet(str(f))
    print(f"\n[{f.name}] shape={df.shape}")
    pick = [c for c in df.columns if c.startswith("여자") and c.endswith("생활인구수")][:4] \
         + [c for c in df.columns if c.startswith("남자") and c.endswith("생활인구수")][:4]
    print(df.select(["행정동코드","시간대라벨"] + pick).sample(n=min(5, df.height), seed=42))
    print(df.select([
        pl.sum("여자50대_생활인구수").alias("여50"), pl.sum("여자60대_생활인구수").alias("여60"), pl.sum("여자70대이상_생활인구수").alias("여70+"),
        pl.sum("남자50대_생활인구수").alias("남50"), pl.sum("남자60대_생활인구수").alias("남60"), pl.sum("남자70대이상_생활인구수").alias("남70+"),
    ]))



[POP_Q_SLOT_NORM_202307.parquet] shape=(315456, 23)
shape: (5, 10)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ 행정동코  ┆ 시간대라  ┆ 여자10대  ┆ 여자10대_ ┆ … ┆ 남자10대  ┆ 남자10대_ ┆ 남자20대_ ┆ 남자30대 │
│ 드        ┆ 벨        ┆ 미만_생활 ┆ 생활인구  ┆   ┆ 미만_생활 ┆ 생활인구  ┆ 생활인구  ┆ _생활인  │
│ ---       ┆ ---       ┆ 인구수    ┆ 수        ┆   ┆ 인구수    ┆ 수        ┆ 수        ┆ 구수     │
│ str       ┆ str       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│           ┆           ┆ f64       ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 11650581  ┆ 04_08     ┆ 1396.2631 ┆ 1963.1057 ┆ … ┆ 1656.5072 ┆ 2172.5355 ┆ 1292.1016 ┆ 1641.092 │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ 8        │
│ 11230610  ┆ 20_24     ┆ 843.8586  ┆ 896.1272  ┆ … ┆ 831.7121  ┆ 877.8912  ┆ 1224.63

# 파생변수 생성

## 1차 파생 변수 생성 (각 그룹 내)

### 생활인구 기반 파생변수 생성 함수 - "행정동코드", "기준년분기_코드"

In [368]:
import polars as pl
from pathlib import Path

def make_features_pop_wide(src_path, out_path=None):
    df = pl.read_parquet(str(src_path))

    # 1️⃣ 기본 컬럼 체크
    base_cols = ["행정동코드", "기준년분기_코드", "시간대라벨", "총생활인구수"]
    for c in base_cols:
        if c not in df.columns:
            raise ValueError(f"❌ 필수 컬럼 누락: {c}")

    # 2️⃣ 남녀별 컬럼 자동 탐색
    male_cols = [c for c in df.columns if c.startswith("남자")]
    female_cols = [c for c in df.columns if c.startswith("여자")]

    # 3️⃣ 남녀별 생활인구 합산 → materialize
    df = df.with_columns([
        pl.sum_horizontal(male_cols).alias("남자생활인구수"),
        pl.sum_horizontal(female_cols).alias("여자생활인구수"),
    ])
    
    # ⚠️ 총생활인구수가 null인 경우에만 남녀합으로 대체
    df = df.with_columns([
        pl.when(pl.col("총생활인구수").is_null() | (pl.col("총생활인구수") == 0))
          .then(pl.col("남자생활인구수") + pl.col("여자생활인구수"))
          .otherwise(pl.col("총생활인구수"))
          .alias("총생활인구수_보정")
    ])
    
    # 4️⃣ 중복 제거: 동일한 (행정동, 분기, 시간대)에 대해 한 줄만 남기기
    agg_df = (
        df.group_by(["행정동코드", "기준년분기_코드", "시간대라벨"])
          .agg([
              pl.col("총생활인구수_보정").mean().alias("총생활인구수"),  # 동일값이라 mean 사용
              pl.col("남자생활인구수").sum().alias("남자생활인구수"),
              pl.col("여자생활인구수").sum().alias("여자생활인구수"),
          ])
    )


    # 5️⃣ 주간/야간 비율 (시간대라벨 형태 맞춰 수정)
    time_ratio = (
        agg_df.with_columns([
            pl.when(pl.col("시간대라벨").is_in(["06~11", "11~14", "14~17", "17~21"]))
              .then(pl.col("총생활인구수")).otherwise(0).alias("주간인구"),
            pl.when(pl.col("시간대라벨").is_in(["21~24", "00~06"]))
              .then(pl.col("총생활인구수")).otherwise(0).alias("야간인구")
        ])
        .group_by(["행정동코드", "기준년분기_코드"])
        .agg([
            pl.sum("주간인구").alias("주간생활인구"),
            pl.sum("야간인구").alias("야간생활인구"),
            pl.sum("총생활인구수").alias("총생활인구수")
        ])
        .with_columns([
            (pl.col("주간생활인구") / pl.col("총생활인구수")).fill_nan(0).fill_null(0).alias("주간생활비율"),
            (pl.col("야간생활인구") / pl.col("총생활인구수")).fill_nan(0).fill_null(0).alias("야간생활비율")
        ])
    )


    # 6️⃣ 남녀비율
    sex_ratio = (
        agg_df.group_by(["행정동코드", "기준년분기_코드"])
        .agg([
            pl.sum("남자생활인구수").alias("남자합"),
            pl.sum("여자생활인구수").alias("여자합"),
        ])
        .with_columns([
            (pl.col("남자합") / pl.col("여자합")).fill_nan(0).fill_null(0).alias("남녀비율")
        ])
        .select(["행정동코드", "기준년분기_코드", "남녀비율"])
    )

    # 7️⃣ 고령화비율 (60대이상+70대이상) / (20~40대)
    df_raw = pl.read_parquet(str(src_path))  # 원본 재읽기
    all_cols = df_raw.columns
    
    old_cols = [c for c in all_cols if any(x in c for x in ["60대", "70대이상"])]
    young_cols = [c for c in all_cols if any(x in c for x in ["20대", "30대", "40대"])]
    
    if not old_cols or not young_cols:
        print(f"⚠️ 연령대 컬럼 탐색 실패 old:{old_cols}, young:{young_cols}")
        df_raw = df_raw.with_columns([
            pl.lit(0).alias("고령층"),
            pl.lit(0).alias("청장년층")
        ])
    else:
        df_raw = df_raw.with_columns([
            pl.sum_horizontal(old_cols).alias("고령층"),
            pl.sum_horizontal(young_cols).alias("청장년층")
        ])
    
    age_ratio = (
        df_raw.group_by(["행정동코드", "기준년분기_코드"])
        .agg([
            pl.col("고령층").sum().alias("고령층합"),
            pl.col("청장년층").sum().alias("청장년층합")
        ])
        .with_columns([
            (pl.col("고령층합") / pl.col("청장년층합")).fill_nan(0).fill_null(0).alias("고령화비율")
        ])
        .select(["행정동코드", "기준년분기_코드", "고령화비율"])
    )
    

    # 8️⃣ 생활인구 표준편차
    var_pop = (
        agg_df.group_by(["행정동코드", "기준년분기_코드"])
        .agg(pl.col("총생활인구수").std().alias("생활인구표준편차"))
    )

    # 9️⃣ 병합
    features = (
        time_ratio
        .join(sex_ratio, on=["행정동코드", "기준년분기_코드"], how="left")
        .join(age_ratio, on=["행정동코드", "기준년분기_코드"], how="left")
        .join(var_pop, on=["행정동코드", "기준년분기_코드"], how="left")
        .fill_nan(0)
        .fill_null(0)
    )
     
    if out_path:
        Path(out_path).parent.mkdir(parents=True, exist_ok=True)
        features.write_parquet(out_path)
        print(f"✅ 저장 완료: {out_path}")
        # 중복 키 검증
        dup = (
            features.group_by(["행정동코드", "기준년분기_코드"])
            .count()
            .filter(pl.col("count") > 1)
        )
        print(f"⚙️ 중복 조합 개수: {dup.height}")

    return features



def make_features_pop_all_wide(src_dir, out_path):
    files = sorted(Path(src_dir).glob("POP_Q_SLOT_NORM_??????.parquet"))
    if not files:
        raise FileNotFoundError(f"❌ 대상 파일 없음: {src_dir}")

    all_feats = []
    for f in files:
        print(f"📊 처리 중: {f.name}")
        feats = make_features_pop_wide(f)  # 위에서 만든 단일파일 함수 재사용
        feats = feats.with_columns(pl.lit(f.stem).alias("파일명"))  # 출처 추적용
        all_feats.append(feats)

    df_all = pl.concat(all_feats, how="diagonal_relaxed")
    Path(out_path).parent.mkdir(parents=True, exist_ok=True)
    df_all.write_parquet(out_path)
    print(f"✅ 전체 병합 완료 → {out_path}")
    return df_all

make_features_pop_all_wide(
    src_dir=r"C:\Users\user\OneDrive\dataset\output\pop_q_parquet",
    out_path=r"C:\Users\user\OneDrive\dataset\output\features\features_pop.parquet"
)


📊 처리 중: POP_Q_SLOT_NORM_201901.parquet
📊 처리 중: POP_Q_SLOT_NORM_201902.parquet
📊 처리 중: POP_Q_SLOT_NORM_201903.parquet
📊 처리 중: POP_Q_SLOT_NORM_201904.parquet
📊 처리 중: POP_Q_SLOT_NORM_201905.parquet
📊 처리 중: POP_Q_SLOT_NORM_201906.parquet
📊 처리 중: POP_Q_SLOT_NORM_201907.parquet
📊 처리 중: POP_Q_SLOT_NORM_201908.parquet
📊 처리 중: POP_Q_SLOT_NORM_201909.parquet
📊 처리 중: POP_Q_SLOT_NORM_201910.parquet
📊 처리 중: POP_Q_SLOT_NORM_201911.parquet
📊 처리 중: POP_Q_SLOT_NORM_201912.parquet
📊 처리 중: POP_Q_SLOT_NORM_202001.parquet
📊 처리 중: POP_Q_SLOT_NORM_202002.parquet
📊 처리 중: POP_Q_SLOT_NORM_202003.parquet
📊 처리 중: POP_Q_SLOT_NORM_202004.parquet
📊 처리 중: POP_Q_SLOT_NORM_202005.parquet
📊 처리 중: POP_Q_SLOT_NORM_202006.parquet
📊 처리 중: POP_Q_SLOT_NORM_202007.parquet
📊 처리 중: POP_Q_SLOT_NORM_202008.parquet
📊 처리 중: POP_Q_SLOT_NORM_202009.parquet
📊 처리 중: POP_Q_SLOT_NORM_202010.parquet
📊 처리 중: POP_Q_SLOT_NORM_202011.parquet
📊 처리 중: POP_Q_SLOT_NORM_202012.parquet
📊 처리 중: POP_Q_SLOT_NORM_202101.parquet
📊 처리 중: POP_Q_SLOT_NORM_2

행정동코드,기준년분기_코드,주간생활인구,야간생활인구,총생활인구수,주간생활비율,야간생활비율,남녀비율,고령화비율,생활인구표준편차,파일명
str,i32,f64,f64,f64,f64,f64,f64,f64,f64,str
"""11215847""",20191,111971.141205,54908.249579,166879.390784,0.67097,0.32903,0.948907,0.425793,3062.763649,"""POP_Q_SLOT_NORM_201901"""
"""11140635""",20191,59373.201963,34629.312532,94002.514495,0.631613,0.368387,0.816472,0.708304,1845.278604,"""POP_Q_SLOT_NORM_201901"""
"""11470620""",20191,63413.098656,33509.828812,96922.927468,0.654263,0.345737,0.881202,0.535036,785.483471,"""POP_Q_SLOT_NORM_201901"""
"""11380625""",20191,168555.471069,97812.518243,266367.989311,0.632792,0.367208,0.812744,0.569461,5411.403624,"""POP_Q_SLOT_NORM_201901"""
"""11740540""",20191,76029.904439,37866.650781,113896.55522,0.667535,0.332465,0.877191,0.628618,376.962075,"""POP_Q_SLOT_NORM_201901"""
…,…,…,…,…,…,…,…,…,…,…
"""11140605""",20244,79865.003315,17427.538166,97292.541481,0.820875,0.179125,1.247889,0.386509,8913.922068,"""POP_Q_SLOT_NORM_202412"""
"""11350695""",20244,180294.132126,88279.754498,268573.886623,0.671302,0.328698,0.784049,0.529026,1219.552607,"""POP_Q_SLOT_NORM_202412"""
"""11230660""",20244,112473.625855,59069.248948,171542.874803,0.655659,0.344341,0.914253,0.455224,1148.160274,"""POP_Q_SLOT_NORM_202412"""


In [350]:
#중복확인
df = pl.read_parquet(r"C:\Users\user\OneDrive\dataset\output\features\features_pop.parquet")

dup = (
    df.group_by(["행정동_코드", "기준년분기_코드"])
      .len()
      .filter(pl.col("len") > 1)
)
print(f"중복 키 개수: {dup.height}")


중복 키 개수: 0


### 점포 기반 파생변수 생성 함수 - "행정동_코드", "기준_년분기_코드"

In [352]:
import polars as pl
from pathlib import Path

def _entropy(df, keys, cat_col, weight_col, out_col):
    return (
        df.group_by(keys + [cat_col])
          .agg(pl.col(weight_col).sum().alias("cnt"))
          .with_columns((pl.col("cnt") / pl.col("cnt").sum().over(keys)).alias("p"))
          .with_columns((-pl.col("p") * pl.col("p").log()).alias("term"))
          .group_by(keys)
          .agg(pl.col("term").sum().alias(out_col))
    )

def _max_share(df, keys, cat_col, weight_col, out_col):
    return (
        df.group_by(keys + [cat_col])
          .agg(pl.col(weight_col).sum().alias("cnt"))
          .group_by(keys)
          .agg((pl.col("cnt") / pl.col("cnt").sum()).max().alias(out_col))
    )

def make_features_store_refined(src_csv_path, out_parquet_path):
    df = pl.read_csv(src_csv_path)
    df = df.with_columns([
        pl.col(["점포_수","개업_점포_수","폐업_점포_수","프랜차이즈_점포_수"]).cast(pl.Float64),
        pl.col("서비스_업종_코드").str.slice(0,3).alias("대분류코드"),
    ])

    keys = ["행정동_코드","기준_년분기_코드"]

    # 전체 / 대분류 다양도
    ent_total = _entropy(df, keys, "서비스_업종_코드", "점포_수", "업종다양도")
    ent_large = _entropy(df, keys, "대분류코드", "점포_수", "업종다양도_대")

    # 대분류 내부 엔트로피
    inner = (
        df.group_by(keys + ["대분류코드","서비스_업종_코드"])
          .agg(pl.col("점포_수").sum().alias("cnt"))
          .with_columns((pl.col("cnt") / pl.col("cnt").sum().over(keys + ["대분류코드"])).alias("p"))
          .with_columns((-pl.col("p") * pl.col("p").log()).alias("term"))
          .group_by(keys + ["대분류코드"])
          .agg([
              pl.col("term").sum().alias("엔트로피_내부"),
              pl.col("cnt").sum().alias("대분류_점포합")
          ])
    )

    # 가중평균 계산 (대분류별 내부 엔트로피를 점포수 비율로 가중)
    ent_small = (
        inner.group_by(keys)
             .agg([
                 (pl.col("엔트로피_내부") * pl.col("대분류_점포합")).sum().alias("numer"),
                 pl.col("대분류_점포합").sum().alias("denom"),
             ])
             .with_columns(
                 (pl.col("numer") / pl.col("denom")).fill_nan(0).fill_null(0).alias("업종다양도_소")
             )
             .select(keys + ["업종다양도_소"])
    )

    # 대분류 내부가 단일 업종이면 엔트로피_내부가 없기 때문에, 그 경우 전체 엔트로피로 fallback
    ent_small = ent_small.join(ent_total, on=keys, how="left")
    ent_small = ent_small.with_columns(
        pl.when(pl.col("업종다양도_소") == 0)
          .then(pl.col("업종다양도"))
          .otherwise(pl.col("업종다양도_소"))
          .alias("업종다양도_소")
    ).select(keys + ["업종다양도_소"])

    # 최대점유율
    max_large = _max_share(df, keys, "대분류코드", "점포_수", "최대점유율_대")
    max_small = _max_share(df, keys, "서비스_업종_코드", "점포_수", "최대점유율_소")

    # 기본 집계
    base = (
        df.group_by(keys)
          .agg([
              pl.col("점포_수").sum().alias("총점포수"),
              pl.col("개업_점포_수").sum().alias("개업수"),
              pl.col("폐업_점포_수").sum().alias("폐업수"),
              pl.col("프랜차이즈_점포_수").sum().alias("프랜차이즈점포수"),
              pl.col("서비스_업종_코드").n_unique().alias("업종수"),
          ])
          .with_columns([
              (pl.col("폐업수") / (pl.col("개업수")+1e-9)).alias("개폐업률"),
              (pl.col("프랜차이즈점포수") / (pl.col("총점포수")+1e-9)).alias("프랜차이즈비율")
          ])
    )

    feats = (
        base.join(ent_total, on=keys, how="left")
            .join(ent_large, on=keys, how="left")
            .join(ent_small, on=keys, how="left")
            .join(max_large, on=keys, how="left")
            .join(max_small, on=keys, how="left")
            .fill_null(0).fill_nan(0)
    )

    Path(out_parquet_path).parent.mkdir(parents=True, exist_ok=True)
    feats.write_parquet(out_parquet_path)
    print(f"✅ 저장 완료: {out_parquet_path}")
    return feats


make_features_store_refined(
    src_csv_path=r"C:\Users\user\OneDrive\dataset\output\분기집계_점포정보.csv",
    out_parquet_path=r"C:\Users\user\OneDrive\dataset\output\features\features_store.parquet"
)


✅ 저장 완료: C:\Users\user\OneDrive\dataset\output\features\features_store.parquet


행정동_코드,기준_년분기_코드,총점포수,개업수,폐업수,프랜차이즈점포수,업종수,개폐업률,프랜차이즈비율,업종다양도,업종다양도_대,업종다양도_소,최대점유율_대,최대점유율_소
i64,i64,f64,f64,f64,f64,u32,f64,f64,f64,f64,f64,f64,f64
11260610,20241,1246.0,47.0,49.0,109.0,88,1.042553,0.08748,0.0,1.089121,0.0,0.390851,0.155698
11740550,20193,506.0,25.0,11.0,55.0,73,0.44,0.108696,3.697569,1.018264,2.679305,0.498024,0.144269
11215870,20232,2357.0,63.0,65.0,229.0,96,1.031746,0.097157,0.0,0.94976,0.0,0.558761,0.137887
11680610,20203,2856.0,83.0,79.0,237.0,97,0.951807,0.082983,0.0,1.010059,0.0,0.487045,0.12535
11200520,20203,468.0,15.0,15.0,57.0,78,1.0,0.121795,0.0,1.044285,0.0,0.416667,0.106838
…,…,…,…,…,…,…,…,…,…,…,…,…,…
11140645,20224,510.0,10.0,12.0,34.0,75,1.2,0.066667,3.584577,1.008654,2.575923,0.496078,0.180392
11470560,20224,1090.0,45.0,34.0,72.0,78,0.755556,0.066055,3.723398,1.078045,2.645353,0.418349,0.133945
11545670,20193,1953.0,76.0,71.0,204.0,89,0.934211,0.104455,3.840997,1.078843,2.762155,0.40297,0.122888


In [356]:
#중복확인
df = pl.read_parquet(r"C:\Users\user\OneDrive\dataset\output\features\features_store.parquet")

dup = (
    df.group_by(["행정동_코드", "기준_년분기_코드"])
      .len()
      .filter(pl.col("len") > 1)
)
print(f"중복 키 개수: {dup.height}")


중복 키 개수: 0


### 추정매출 기반 파생변수 생성 함수 - "행정동_코드", "기준_년분기_코드"

In [358]:
import polars as pl
from pathlib import Path
import numpy as np

def clean_features_sales(src_path, out_path=None):
    df = pl.read_parquet(src_path)

    # 1️⃣ inf → nan 처리
    df = df.with_columns([
        pl.when(pl.col(c).is_infinite()).then(None).otherwise(pl.col(c)).alias(c)
        for c in df.columns
    ])

    # 2️⃣ NaN/Null → 0 채우기 (특히 비율 계열)
    ratio_cols = [c for c in df.columns if "비율" in c or "균형도" in c]
    df = df.with_columns([
        pl.col(c).fill_nan(0).fill_null(0).alias(c) for c in ratio_cols
    ])

    # 3️⃣ 음수값 허용: 매출증감률은 그대로 둠 (하락 신호 보존)
    #    단, 극단치(-1~+1 범위 이상)는 clipping
    if "매출증감률" in df.columns:
        df = df.with_columns(
            pl.col("매출증감률").clip(-1.0, 1.0).alias("매출증감률")
        )

    # 4️⃣ 로그 변환 (매출규모 / 표준편차 등 분포 안정화)
    log_targets = [c for c in df.columns if any(x in c for x in ["총매출액", "매출표준편차"])]
    for c in log_targets:
        df = df.with_columns(
            pl.when(pl.col(c) > 0)
              .then(pl.col(c).log1p())  # log(1+x)
              .otherwise(0)
              .alias(f"{c}_log")
        )

    # 5️⃣ 비율형(0~1 스케일) clipping
    for c in ratio_cols:
        df = df.with_columns(
            pl.col(c).clip(0, 1).alias(c)
        )

    # 6️⃣ 이상치 완화 (IQR 기반 clipping)
    numeric_cols = [c for c, dtype in zip(df.columns, df.dtypes) if dtype in (pl.Float64, pl.Int64)]
    for c in numeric_cols:
        q1, q3 = df.select(pl.col(c).quantile(0.25)).item(), df.select(pl.col(c).quantile(0.75)).item()
        iqr = q3 - q1
        lower, upper = q1 - 1.5 * iqr, q3 + 1.5 * iqr
        df = df.with_columns(
            pl.col(c).clip(lower, upper).alias(c)
        )

    # 7️⃣ 최종 저장
    if out_path:
        Path(out_path).parent.mkdir(parents=True, exist_ok=True)
        df.write_parquet(out_path)
        print(f"✅ 추정매출 피처 정제 완료 → {out_path}")

    return df


clean_features_sales(
    src_path=r"C:\Users\user\OneDrive\dataset\output\features\features_sales.parquet",
    out_path=r"C:\Users\user\OneDrive\dataset\output\features\features_sales_clean.parquet"
)


✅ 추정매출 피처 정제 완료 → C:\Users\user\OneDrive\dataset\output\features\features_sales_clean.parquet


행정동_코드,기준_년분기_코드,총매출액,주간매출비율,야간매출비율,주중매출비율,주말매출비율,남녀매출비율,성별균형도,연령대다양도,매출표준편차,매출증감률,총매출액_log,매출표준편차_log
i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
11110515,20191,15568061933,0.905312,0.094688,0.772193,0.227807,1.0,0.332735,0.760676,2.3738e8,0.0,23.468487,19.285155
11110515,20192,16874853653,0.930923,0.069077,0.763758,0.236242,0.0,0.340016,0.842339,2.2425e8,0.083941,23.54909,19.228269
11110515,20193,16424404096,0.915983,0.084017,0.80764,0.19236,0.0,0.413488,0.721022,2.4648e8,-0.026694,23.522034,19.322793
11110515,20194,17702770570,0.93797,0.06203,0.805966,0.194034,0.0,0.374179,0.755666,2.5397e8,0.077833,23.596987,19.352728
11110515,20201,15611509280,0.917733,0.082267,0.760563,0.239437,0.942182,0.38252,0.679113,2.2360e8,-0.118132,23.471274,19.225359
…,…,…,…,…,…,…,…,…,…,…,…,…,…
11740700,20234,27657176101,0.91464,0.08536,0.771937,0.228063,1.0,0.266329,0.634797,2.4606e8,-0.078556,24.043151,19.321088
11740700,20241,27507243448,0.89666,0.10334,0.75848,0.24152,0.0,0.280331,0.47384,2.7210e8,-0.005421,24.037715,19.421681
11740700,20242,27029802284,0.910663,0.089337,0.774418,0.225582,0.0,0.340233,0.568373,2.4178e8,-0.017357,24.020206,19.303537


In [360]:
#중복확인
df = pl.read_parquet(r"C:\Users\user\OneDrive\dataset\output\features\features_sales_clean.parquet")

dup = (
    df.group_by(["행정동_코드", "기준_년분기_코드"])
      .len()
      .filter(pl.col("len") > 1)
)
print(f"중복 키 개수: {dup.height}")


중복 키 개수: 0


## 1차 파생 변수 파일 확인

In [370]:
import polars as pl
from pathlib import Path
import os

def preflight_full(pop_path, store_path, sales_path, report_path=None):
    """
    2차 밍글링 전 데이터 사전점검:
    - 경로 및 파일 존재 확인
    - 필수 컬럼 누락 여부
    - 주요 통계(행수, 컬럼수, 결측치)
    - 결과를 콘솔과 리포트 파일로 동시 출력
    """
    print("📋 [사전점검] 시작 -------------------------------")
    logs = []

    def log(msg):
        print(msg)
        logs.append(msg)

    # 파일 존재 확인
    for label, path in {"POP": pop_path, "STORE": store_path, "SALES": sales_path}.items():
        if not os.path.exists(path):
            log(f"❌ {label} 파일 없음: {path}")
            return

    # 요구 컬럼 세트 정의
    need_pop   = {"행정동_코드","기준_년분기_코드","총생활인구수","주간생활비율","야간생활비율","남녀비율","고령화비율","생활인구표준편차"}
    need_store = {"행정동_코드","기준_년분기_코드","총점포수","개업수","폐업수","개폐업률","프랜차이즈비율",
                  "업종다양도","업종다양도_대","업종다양도_소","최대점유율_대","최대점유율_소"}
    need_sales = {"행정동_코드","기준_년분기_코드","총매출액","총매출액_log","주간매출비율","야간매출비율",
                  "주중매출비율","주말매출비율","남녀매출비율","성별균형도","연령대다양도","매출표준편차_log","매출증감률"}

    # 파일별 점검 함수
    def check_dataset(label, path, need_cols):
        df = pl.read_parquet(path)
        # 컬럼명 교정
        rename_map = {}
        if "행정동코드" in df.columns: rename_map["행정동코드"] = "행정동_코드"
        if "기준년분기_코드" in df.columns: rename_map["기준년분기_코드"] = "기준_년분기_코드"
        if rename_map:
            df = df.rename(rename_map)

        miss = need_cols - set(df.columns)
        log(f"\n[{label}] -------------------------------")
        log(f"• 경로: {path}")
        log(f"• Shape: {df.shape}")
        log(f"• 컬럼수: {len(df.columns)}")
        if miss:
            log(f"❌ 누락 컬럼: {sorted(miss)}")
        else:
            log(f"✅ 필수 컬럼 전체 존재")

        # 간단한 결측치 요약 (상위 5개)
        na_summary = df.null_count().to_dicts()[0]
        top_na = sorted(na_summary.items(), key=lambda x: x[1], reverse=True)[:5]
        log("• 결측치 Top5:")
        for c, n in top_na:
            log(f"  {c}: {n:,}")

        return df, miss

    # 세부 점검 실행
    pop_df, miss_pop = check_dataset("POP", pop_path, need_pop)
    store_df, miss_store = check_dataset("STORE", store_path, need_store)
    sales_df, miss_sales = check_dataset("SALES", sales_path, need_sales)

    # 요약 결과
    log("\n📊 [요약 결과]")
    log(f"POP   → {'OK' if not miss_pop else 'MISSING'}")
    log(f"STORE → {'OK' if not miss_store else 'MISSING'}")
    log(f"SALES → {'OK' if not miss_sales else 'MISSING'}")

    # 리포트 저장
    if report_path is None:
        report_path = str(Path(pop_path).parent / "preflight_report.txt")
    with open(report_path, "w", encoding="utf-8") as f:
        f.write("\n".join(logs))
    print(f"\n🧾 리포트 저장 완료 → {report_path}")
    print("📋 [사전점검] 종료 -------------------------------")

    return logs

preflight_full(
    pop_path   = r"C:\Users\user\OneDrive\dataset\output\features\features_pop.parquet",
    store_path = r"C:\Users\user\OneDrive\dataset\output\features\features_store.parquet",
    sales_path = r"C:\Users\user\OneDrive\dataset\output\features\features_sales_clean.parquet"
)


📋 [사전점검] 시작 -------------------------------

[POP] -------------------------------
• 경로: C:\Users\user\OneDrive\dataset\output\features\features_pop.parquet
• Shape: (30528, 11)
• 컬럼수: 11
✅ 필수 컬럼 전체 존재
• 결측치 Top5:
  행정동_코드: 0
  기준_년분기_코드: 0
  주간생활인구: 0
  야간생활인구: 0
  총생활인구수: 0

[STORE] -------------------------------
• 경로: C:\Users\user\OneDrive\dataset\output\features\features_store.parquet
• Shape: (10200, 14)
• 컬럼수: 14
✅ 필수 컬럼 전체 존재
• 결측치 Top5:
  행정동_코드: 0
  기준_년분기_코드: 0
  총점포수: 0
  개업수: 0
  폐업수: 0

[SALES] -------------------------------
• 경로: C:\Users\user\OneDrive\dataset\output\features\features_sales_clean.parquet
• Shape: (10168, 14)
• 컬럼수: 14
✅ 필수 컬럼 전체 존재
• 결측치 Top5:
  행정동_코드: 0
  기준_년분기_코드: 0
  총매출액: 0
  주간매출비율: 0
  야간매출비율: 0

📊 [요약 결과]
POP   → OK
STORE → OK
SALES → OK

🧾 리포트 저장 완료 → C:\Users\user\OneDrive\dataset\output\features\preflight_report.txt
📋 [사전점검] 종료 -------------------------------


['\n[POP] -------------------------------',
 '• 경로: C:\\Users\\user\\OneDrive\\dataset\\output\\features\\features_pop.parquet',
 '• Shape: (30528, 11)',
 '• 컬럼수: 11',
 '✅ 필수 컬럼 전체 존재',
 '• 결측치 Top5:',
 '  행정동_코드: 0',
 '  기준_년분기_코드: 0',
 '  주간생활인구: 0',
 '  야간생활인구: 0',
 '  총생활인구수: 0',
 '\n[STORE] -------------------------------',
 '• 경로: C:\\Users\\user\\OneDrive\\dataset\\output\\features\\features_store.parquet',
 '• Shape: (10200, 14)',
 '• 컬럼수: 14',
 '✅ 필수 컬럼 전체 존재',
 '• 결측치 Top5:',
 '  행정동_코드: 0',
 '  기준_년분기_코드: 0',
 '  총점포수: 0',
 '  개업수: 0',
 '  폐업수: 0',
 '\n[SALES] -------------------------------',
 '• 경로: C:\\Users\\user\\OneDrive\\dataset\\output\\features\\features_sales_clean.parquet',
 '• Shape: (10168, 14)',
 '• 컬럼수: 14',
 '✅ 필수 컬럼 전체 존재',
 '• 결측치 Top5:',
 '  행정동_코드: 0',
 '  기준_년분기_코드: 0',
 '  총매출액: 0',
 '  주간매출비율: 0',
 '  야간매출비율: 0',
 '\n📊 [요약 결과]',
 'POP   → OK',
 'STORE → OK',
 'SALES → OK']

## [A1] 2차 파생 변수 생성
그룹 간 - 생활인구 × 점포 / 점포 × 매출 / 생활인구 × 매출

In [372]:
import polars as pl
from pathlib import Path

def make_mingle_stage2(pop_path: str, store_path: str, sales_clean_path: str, out_path: str):
    EPS = 1e-9

    # 1) 로드
    pop = pl.read_parquet(pop_path)
    store = pl.read_parquet(store_path)
    sales = pl.read_parquet(sales_clean_path)

    # 1-1) 키 표준화: 컬럼명 정리
    if "행정동코드" in pop.columns: pop = pop.rename({"행정동코드":"행정동_코드"})
    if "기준년분기_코드" in pop.columns: pop = pop.rename({"기준년분기_코드":"기준_년분기_코드"})

    # 1-2) 키 타입 통일: 모두 Utf8로(조인 안정)
    def force_key_str(df):
        cols = df.columns
        if "행정동_코드" in cols:
            df = df.with_columns(pl.col("행정동_코드").cast(pl.Utf8))
        if "기준_년분기_코드" in cols:
            df = df.with_columns(pl.col("기준_년분기_코드").cast(pl.Utf8))
        return df

    pop   = force_key_str(pop)
    store = force_key_str(store)
    sales = force_key_str(sales)

    # 1-3) 필요한 컬럼만 얇게 선택
    pop = pop.select([c for c in [
        "행정동_코드","기준_년분기_코드","총생활인구수","주간생활비율","야간생활비율","남녀비율","고령화비율","생활인구표준편차"
    ] if c in pop.columns])

    store = store.select([c for c in [
        "행정동_코드","기준_년분기_코드","총점포수","개업수","폐업수","개폐업률","프랜차이즈비율",
        "업종다양도","업종다양도_대","업종다양도_소","최대점유율_대","최대점유율_소"
    ] if c in store.columns])

    sales = sales.select([c for c in [
        "행정동_코드","기준_년분기_코드","총매출액","총매출액_log","주간매출비율","야간매출비율",
        "주중매출비율","주말매출비율","남녀매출비율","성별균형도","연령대다양도","매출표준편차_log","매출증감률"
    ] if c in sales.columns])

    # 2) 병합
    df = (pop.join(store, on=["행정동_코드","기준_년분기_코드"], how="inner")
             .join(sales, on=["행정동_코드","기준_년분기_코드"], how="inner")
             .fill_nan(0).fill_null(0))

    # 3) 2차 밍글링
    df = df.with_columns([
        (pl.col("주간생활비율") * pl.col("주간매출비율")).alias("체류활성도_주간"),
        (pl.col("야간생활비율") * pl.col("야간매출비율")).alias("체류활성도_야간"),
        (pl.col("총매출액_log") / (pl.col("총점포수")+1)).alias("점포생산성_log"),
        (pl.col("총매출액_log") / (pl.col("총생활인구수")+1)).alias("수요당매출_log"),
        (pl.col("총점포수") / (pl.col("총생활인구수")+1)).alias("상권포화도"),
        (pl.col("업종다양도") * pl.col("연령대다양도")).alias("다양성시너지"),
        (pl.col("최대점유율_대") * pl.col("총매출액_log")).alias("집중성과_대"),
        (pl.col("최대점유율_소") * pl.col("총매출액_log")).alias("집중성과_소"),
        (pl.col("주중매출비율") * pl.col("주간매출비율")).alias("출퇴근형지수"),
        (pl.col("주말매출비율") * pl.col("야간매출비율")).alias("주말야간형지수"),
        (pl.col("주중매출비율") - pl.col("주말매출비율")).abs().alias("주중주말격차"),
        (1 - pl.col("성별균형도")).clip(0,1).alias("성별밸런스지수"),
        (1 - (pl.col("남녀매출비율") - 1).abs()).clip(0,1).alias("남녀비율근접도"),
        ((1 - pl.col("성별균형도")).clip(0,1) * (1 - (pl.col("남녀매출비율") - 1).abs()).clip(0,1)).alias("성별정합도"),
        (pl.col("매출표준편차_log") * pl.col("매출증감률")).abs().alias("매출민감도_raw"),
        (pl.col("프랜차이즈비율") * pl.col("개폐업률")).alias("프랜차이즈지배력"),
        (pl.col("개폐업률") * (1/((pl.col("총점포수")/(pl.col("총생활인구수")+1))+EPS))).alias("개폐업탄력도"),
        (pl.col("고령화비율") * pl.col("총매출액_log")).alias("고령층영향도_log"),
    ])

    # 4) (선택) 기준_년분기_코드만 숫자로 되돌리기
    if df["기준_년분기_코드"].dtype == pl.Utf8:
        df = df.with_columns(pl.col("기준_년분기_코드").cast(pl.Int32).alias("기준_년분기_코드"))

    # 5) 저장
    Path(out_path).parent.mkdir(parents=True, exist_ok=True)
    df.write_parquet(out_path)
    print(f"✅ 2차 밍글링 완료 → {out_path}")
    return df


make_mingle_stage2(
    pop_path   = r"C:\Users\user\OneDrive\dataset\output\features\features_pop.parquet",
    store_path = r"C:\Users\user\OneDrive\dataset\output\features\features_store.parquet",
    sales_clean_path = r"C:\Users\user\OneDrive\dataset\output\features\features_sales_clean.parquet",
    out_path  = r"C:\Users\user\OneDrive\dataset\output\features\features_mingle_stage2.parquet"
)


✅ 2차 밍글링 완료 → C:\Users\user\OneDrive\dataset\output\features\features_mingle_stage2.parquet


행정동_코드,기준_년분기_코드,총생활인구수,주간생활비율,야간생활비율,남녀비율,고령화비율,생활인구표준편차,총점포수,개업수,폐업수,개폐업률,프랜차이즈비율,업종다양도,업종다양도_대,업종다양도_소,최대점유율_대,최대점유율_소,총매출액,총매출액_log,주간매출비율,야간매출비율,주중매출비율,주말매출비율,남녀매출비율,성별균형도,연령대다양도,매출표준편차_log,매출증감률,체류활성도_주간,체류활성도_야간,점포생산성_log,수요당매출_log,상권포화도,다양성시너지,집중성과_대,집중성과_소,출퇴근형지수,주말야간형지수,주중주말격차,성별밸런스지수,남녀비율근접도,성별정합도,매출민감도_raw,프랜차이즈지배력,개폐업탄력도,고령층영향도_log
str,i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""11215847""",20191,166879.390784,0.67097,0.32903,0.948907,0.425793,3062.763649,1356.0,66.0,52.0,0.787879,0.089971,3.822574,1.091338,2.731236,0.379056,0.100295,36940035334,24.332562,0.83806,0.16194,0.680575,0.319425,1.0,0.375103,0.904068,19.610515,0.0,0.562314,0.053283,0.017931,0.000146,0.008126,3.455865,9.223405,2.440434,0.570363,0.051728,0.361149,0.624897,1.0,0.624897,0.0,0.070886,96.962761,10.360643
"""11140635""",20191,94002.514495,0.631613,0.368387,0.816472,0.708304,1845.278604,598.0,21.0,15.0,0.714286,0.100334,0.0,1.043797,0.0,0.421405,0.098662,29189437749,24.097073,0.881433,0.118567,0.771025,0.228975,1.0,0.260035,1.023676,19.815624,0.0,0.556724,0.043679,0.040229,0.000256,0.006361,0.0,10.154619,2.37747,0.679606,0.027149,0.542049,0.739965,1.0,0.739965,0.0,0.071667,112.283206,17.068058
"""11470620""",20191,96922.927468,0.654263,0.345737,0.881202,0.535036,785.483471,944.0,44.0,41.0,0.931818,0.086864,3.769616,1.023835,2.745781,0.520127,0.09322,25140461349,23.947744,0.82439,0.17561,0.732295,0.267705,1.0,0.288232,0.775596,19.339853,0.0,0.539368,0.060715,0.025342,0.000247,0.00974,2.923697,12.455871,2.232417,0.603696,0.047012,0.464589,0.711768,1.0,0.711768,0.0,0.080942,95.673166,12.812913
"""11380625""",20191,266367.989311,0.632792,0.367208,0.812744,0.569461,5411.403624,1761.0,78.0,51.0,0.653846,0.084611,3.801938,1.067037,2.734901,0.431005,0.092561,28646143992,24.078285,0.838154,0.161846,0.775809,0.224191,0.0,0.330294,0.915919,19.311944,0.0,0.530377,0.059431,0.013665,0.00009,0.006611,3.482266,10.377864,2.228711,0.650247,0.036285,0.551618,0.669706,0.0,0.0,0.0,0.055323,98.900802,13.711634
"""11740540""",20191,113896.55522,0.667535,0.332465,0.877191,0.628618,376.962075,903.0,45.0,39.0,0.866667,0.131783,0.0,0.96271,0.0,0.537099,0.219269,36458017317,24.319427,0.884156,0.115844,0.728311,0.271689,1.0,0.286175,1.045493,19.695097,0.0,0.590205,0.038514,0.026902,0.000214,0.007928,0.0,13.061929,5.332499,0.643941,0.031474,0.456623,0.713825,1.0,0.713825,0.0,0.114212,109.314731,15.287632
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""11140605""",20244,97292.541481,0.820875,0.179125,1.247889,0.386509,8913.922068,2864.0,57.0,44.0,0.77193,0.046788,3.366529,0.944009,2.42252,0.598464,0.174581,118265118450,26.23267,0.853117,0.146883,0.856773,0.143227,0.0,0.371156,0.7376,21.770797,0.056538,0.700303,0.02631,0.009156,0.00027,0.029437,2.483153,15.699301,4.579726,0.730928,0.021038,0.713545,0.628844,0.0,0.0,1.230878,0.036117,26.223388,10.139162
"""11350695""",20244,268573.886623,0.671302,0.328698,0.784049,0.529026,1219.552607,1485.0,33.0,37.0,1.121212,0.133333,3.777988,0.98607,2.791918,0.544108,0.074074,103017666325,25.358166,0.849611,0.150389,0.744435,0.255565,0.0,0.309421,1.096429,20.859323,0.004299,0.570345,0.049433,0.017065,0.000094,0.005529,4.142294,13.797575,1.878383,0.63248,0.038434,0.488871,0.690579,0.0,0.0,0.089677,0.149495,202.780716,13.415124
"""11230660""",20244,171542.874803,0.655659,0.344341,0.914253,0.455224,1148.160274,1980.0,51.0,70.0,1.372549,0.124747,3.83186,1.064695,2.767166,0.395455,0.112121,97152708951,25.29955,0.89977,0.10023,0.742678,0.257322,1.0,0.272992,0.805598,20.515419,0.017477,0.589942,0.034513,0.012771,0.000147,0.011542,3.086937,10.004822,2.836616,0.66824,0.025791,0.485356,0.727008,1.0,0.727008,0.358554,0.171222,118.915332,11.516952


In [384]:
# 중복확인
df = pl.read_parquet(r"C:\Users\user\OneDrive\dataset\output\features\features_mingle_stage2.parquet")

# ① 구조적 중복: 동일 행정동×분기 조합이 여러 개인 그룹
dup_len = (
    df.group_by(["행정동_코드","기준_년분기_코드"])
      .len()
      .filter(pl.col("len") > 1)
)

# ② 내용적 완전중복: 주요값이 모두 동일한 행의 개수
subset_cols = ["행정동_코드","기준_년분기_코드","총점포수","총생활인구수","총매출액"]
dup_content = df.height - df.unique(subset=subset_cols).height

print(f"구조적 중복 건수 : {dup_len.height:,}")
print(f"내용적 완전중복 건수 : {dup_content:,}")


구조적 중복 건수 : 10,008
내용적 완전중복 건수 : 0


QC

In [386]:
import polars as pl

m2 = pl.read_parquet(r"C:\Users\user\OneDrive\dataset\output\features\features_mingle_stage2.parquet")

# 1) 필수 컬럼 존재 확인
must = {
    "체류활성도_주간","체류활성도_야간","점포생산성_log","수요당매출_log","상권포화도",
    "다양성시너지","집중성과_대","집중성과_소","출퇴근형지수","주말야간형지수",
    "주중주말격차","성별밸런스지수","남녀비율근접도","성별정합도","매출민감도_raw",
    "프랜차이즈지배력","개폐업탄력도","고령층영향도_log"
}
missing = sorted(list(must - set(m2.columns)))
print("❌ Missing:", missing) if missing else print("✅ 필수 밍글링 컬럼 존재")

# 2) 결측치(NA) 총합
na_by_col = m2.select([pl.col(c).is_null().sum().alias(c) for c in m2.columns])
na_total = int(na_by_col.select(pl.sum_horizontal(pl.all())).to_series().item())
print(f"NA 총합: {na_total:,}", "(있음)" if na_total else "(없음)")

# 3) ∞(inf) 존재 여부 — float 컬럼만 검사
float_cols = [c for c, dt in zip(m2.columns, m2.dtypes) if dt in (pl.Float64, pl.Float32)]
inf_by_col = m2.select([pl.col(c).is_infinite().sum().alias(c) for c in float_cols])
inf_total = int(inf_by_col.select(pl.sum_horizontal(pl.all())).to_series().item())
print("∞ 총합:", inf_total, "→", "있음" if inf_total > 0 else "없음")

# (선택) inf가 있는 컬럼 목록
if inf_total > 0:
    inf_cols = [c for c, v in zip(inf_by_col.columns, inf_by_col.row(0)) if v > 0]
    print("∞ 존재 컬럼:", inf_cols)

# 4) shape
print("shape:", m2.shape)


✅ 필수 밍글링 컬럼 존재
NA 총합: 0 (없음)
∞ 총합: 0 → 없음
shape: (30024, 47)


## [A2] 3차 파생 변수 생성
4개 지수 신규 생성 - 상권활성지수, 소비균형성지수, 상권지속가능지수, 매출민감성지수

In [394]:
import polars as pl
from pathlib import Path

def make_mingle_stage3(in_path: str, out_path: str):
    EPS = 1e-9
    df = pl.read_parquet(in_path)

    # 필요 컬럼 점검
    need = {
        "행정동_코드","기준_년분기_코드",
        "총매출액_log","점포생산성_log","수요당매출_log","상권포화도",
        "체류활성도_주간","체류활성도_야간","다양성시너지",
        "주간매출비율","야간매출비율","주중주말격차",
        "성별균형도","남녀비율근접도","연령대다양도",
        "매출표준편차_log","개폐업률","프랜차이즈지배력","매출증감률"
    }
    miss = need - set(df.columns)
    if miss:
        raise ValueError(f"❌ 3차 지수화에 필요한 컬럼 누락: {sorted(miss)}")

    # percent-rank 유틸 (분기 내 0~1)
    def pct_rank(expr: pl.Expr) -> pl.Expr:
        r = expr.rank(method="average").over("기준_년분기_코드")
        n = pl.count().over("기준_년분기_코드")
        return ((r - 1) / (n - 1 + EPS)).fill_nan(0).fill_null(0).clip(0, 1)

    # 균형도 보조 파생
    df = df.with_columns([
        (1 - (pl.col("주간매출비율") - pl.col("야간매출비율")).abs()).clip(0, 1).alias("시간대균형도"),
        (1 - pl.col("주중주말격차")).clip(0, 1).alias("주중주말균형도"),
        pct_rank(pl.col("연령대다양도")).alias("연령다양도_norm"),
    ])

    # 분기별 상대화(높을수록 좋게/강하게)
    df = df.with_columns([
        # 활성 구성요소
        pct_rank(pl.col("총매출액_log")).alias("규모P"),
        pct_rank(pl.col("점포생산성_log")).alias("생산성P"),
        pct_rank(pl.col("수요당매출_log")).alias("효율P"),
        ((pct_rank(pl.col("체류활성도_주간")) + pct_rank(pl.col("체류활성도_야간"))) / 2).alias("체류P"),
        pct_rank(pl.col("다양성시너지")).alias("다양성P"),
        (1 - pct_rank(pl.col("상권포화도"))).alias("포화역P"),

        # 지속 구성요소
        (1 - pct_rank(pl.col("매출표준편차_log"))).alias("변동안정P"),
        (1 - pct_rank(pl.col("개폐업률"))).alias("개폐안정P"),
        (1 - pct_rank(pl.col("프랜차이즈지배력"))).alias("과점완화P"),
    ])

    # 지수 산출
    df = df.with_columns([
        # 활성지수
        (0.25*pl.col("규모P") + 0.20*pl.col("생산성P") + 0.20*pl.col("효율P")
         + 0.15*pl.col("체류P") + 0.10*pl.col("다양성P") + 0.10*pl.col("포화역P")).alias("활성지수"),

        # 소비균형성지수
        ((pl.col("성별균형도").clip(0,1) + pl.col("남녀비율근접도").clip(0,1)
          + pl.col("시간대균형도") + pl.col("주중주말균형도") + pl.col("연령다양도_norm"))/5).alias("소비균형성지수"),
        
        # 상권지속가능지수
        ((pl.col("변동안정P") + pl.col("개폐안정P") + pl.col("포화역P") + pl.col("과점완화P"))/4).alias("상권지속가능지수"),
    ])

    # 🔁 매출민감성지수 (신규) — ‘민감도’ 개념에 맞춰 재정의
    df = df.with_columns([
        pct_rank(pl.col("매출표준편차_log")).alias("민감도P_변동"),
        pct_rank(pl.col("매출증감률").abs()).alias("민감도P_증감"),
        pct_rank(pl.col("주중주말격차")).alias("민감도P_주중주말"),
        pct_rank((pl.col("주간매출비율") - pl.col("야간매출비율")).abs()).alias("민감도P_시간대"),
    ]).with_columns([
        (0.4*pl.col("민감도P_변동") + 0.4*pl.col("민감도P_증감")
         + 0.1*pl.col("민감도P_주중주말") + 0.1*pl.col("민감도P_시간대")).alias("매출민감성지수"),
    ])

    # 보기 편한 점수
    df = df.with_columns([
        (pl.col("활성지수")*100).round(3).alias("활성지수_100"),
        (pl.col("소비균형성지수")*100).round(3).alias("소비균형성지수_100"),
        (pl.col("상권지속가능지수")*100).round(3).alias("상권지속가능지수_100"),
        (pl.col("매출민감성지수")*100).round(3).alias("매출민감성지수_100"),
    ])

    # 과거 컬럼 정리(있으면 제거)
    drop_legacy = [c for c in ["탄력성지수_내적","탄력성지수_내적_100"] if c in df.columns]
    if drop_legacy:
        df = df.drop(drop_legacy)

    Path(out_path).parent.mkdir(parents=True, exist_ok=True)
    df.write_parquet(out_path)
    print(f"✅ 3차 지수화 완료 → {out_path}")
    return df


make_mingle_stage3(
    in_path = r"C:\Users\user\OneDrive\dataset\output\features\features_mingle_stage2.parquet",
    out_path= r"C:\Users\user\OneDrive\dataset\output\features\features_mingle_stage3.parquet"
)


C:\Users\user\AppData\Local\Temp\ipykernel_6640\2722364337.py:24: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  n = pl.count().over("기준_년분기_코드")


✅ 3차 지수화 완료 → C:\Users\user\OneDrive\dataset\output\features\features_mingle_stage3.parquet


행정동_코드,기준_년분기_코드,총생활인구수,주간생활비율,야간생활비율,남녀비율,고령화비율,생활인구표준편차,총점포수,개업수,폐업수,개폐업률,프랜차이즈비율,업종다양도,업종다양도_대,업종다양도_소,최대점유율_대,최대점유율_소,총매출액,총매출액_log,주간매출비율,야간매출비율,주중매출비율,주말매출비율,남녀매출비율,성별균형도,연령대다양도,매출표준편차_log,매출증감률,체류활성도_주간,체류활성도_야간,점포생산성_log,수요당매출_log,상권포화도,다양성시너지,집중성과_대,집중성과_소,출퇴근형지수,주말야간형지수,주중주말격차,성별밸런스지수,남녀비율근접도,성별정합도,매출민감도_raw,프랜차이즈지배력,개폐업탄력도,고령층영향도_log,시간대균형도,주중주말균형도,연령다양도_norm,규모P,생산성P,효율P,체류P,다양성P,포화역P,변동안정P,개폐안정P,과점완화P,활성지수,소비균형성지수,상권지속가능지수,민감도P_변동,민감도P_증감,민감도P_주중주말,민감도P_시간대,매출민감성지수,활성지수_100,소비균형성지수_100,상권지속가능지수_100,매출민감성지수_100
str,i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""11215847""",20191,166879.390784,0.67097,0.32903,0.948907,0.425793,3062.763649,1356.0,66.0,52.0,0.787879,0.089971,3.822574,1.091338,2.731236,0.379056,0.100295,36940035334,24.332562,0.83806,0.16194,0.680575,0.319425,1.0,0.375103,0.904068,19.610515,0.0,0.562314,0.053283,0.017931,0.000146,0.008126,3.455865,9.223405,2.440434,0.570363,0.051728,0.361149,0.624897,1.0,0.624897,0.0,0.070886,96.962761,10.360643,0.323879,0.638851,0.6872,0.6944,0.2504,0.3568,0.6012,0.8144,0.3208,0.3416,0.308,0.368,0.49874,0.605007,0.3346,0.6584,0.5,0.02,0.1424,0.4796,49.874,60.501,33.46,47.96
"""11140635""",20191,94002.514495,0.631613,0.368387,0.816472,0.708304,1845.278604,598.0,21.0,15.0,0.714286,0.100334,0.0,1.043797,0.0,0.421405,0.098662,29189437749,24.097073,0.881433,0.118567,0.771025,0.228975,1.0,0.260035,1.023676,19.815624,0.0,0.556724,0.043679,0.040229,0.000256,0.006361,0.0,10.154619,2.37747,0.679606,0.027149,0.542049,0.739965,1.0,0.739965,0.0,0.071667,112.283206,17.068058,0.237134,0.457951,0.9008,0.5864,0.7904,0.8248,0.4012,0.1856,0.5808,0.2576,0.4724,0.3512,0.60646,0.571184,0.4155,0.7424,0.5,0.6824,0.6896,0.63416,60.646,57.118,41.55,63.416
"""11470620""",20191,96922.927468,0.654263,0.345737,0.881202,0.535036,785.483471,944.0,44.0,41.0,0.931818,0.086864,3.769616,1.023835,2.745781,0.520127,0.09322,25140461349,23.947744,0.82439,0.17561,0.732295,0.267705,1.0,0.288232,0.775596,19.339853,0.0,0.539368,0.060715,0.025342,0.000247,0.00974,2.923697,12.455871,2.232417,0.603696,0.047012,0.464589,0.711768,1.0,0.711768,0.0,0.080942,95.673166,12.812913,0.351221,0.535411,0.3728,0.5072,0.512,0.7952,0.5212,0.5936,0.1952,0.4832,0.1544,0.2528,0.5453,0.509533,0.2714,0.5168,0.5,0.248,0.0752,0.43904,54.53,50.953,27.14,43.904
"""11380625""",20191,266367.989311,0.632792,0.367208,0.812744,0.569461,5411.403624,1761.0,78.0,51.0,0.653846,0.084611,3.801938,1.067037,2.734901,0.431005,0.092561,28646143992,24.078285,0.838154,0.161846,0.775809,0.224191,0.0,0.330294,0.915919,19.311944,0.0,0.530377,0.059431,0.013665,0.00009,0.006611,3.482266,10.377864,2.228711,0.650247,0.036285,0.551618,0.669706,0.0,0.0,0.0,0.055323,98.900802,13.711634,0.323693,0.448382,0.7208,0.5744,0.1568,0.0568,0.4836,0.836,0.5544,0.4952,0.6056,0.6152,0.3979,0.364634,0.5676,0.5048,0.5,0.7208,0.1448,0.48848,39.79,36.463,56.76,48.848
"""11740540""",20191,113896.55522,0.667535,0.332465,0.877191,0.628618,376.962075,903.0,45.0,39.0,0.866667,0.131783,0.0,0.96271,0.0,0.537099,0.219269,36458017317,24.319427,0.884156,0.115844,0.728311,0.271689,1.0,0.286175,1.045493,19.695097,0.0,0.590205,0.038514,0.026902,0.000214,0.007928,0.0,13.061929,5.332499,0.643941,0.031474,0.456623,0.713825,1.0,0.713825,0.0,0.114212,109.314731,15.287632,0.231689,0.543377,0.9224,0.6896,0.5408,0.6824,0.4964,0.1856,0.3464,0.296,0.2072,0.0776,0.5447,0.596728,0.2318,0.704,0.5,0.2,0.728,0.5744,54.47,59.673,23.18,57.44
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""11140605""",20244,97292.541481,0.820875,0.179125,1.247889,0.386509,8913.922068,2864.0,57.0,44.0,0.77193,0.046788,3.366529,0.944009,2.42252,0.598464,0.174581,118265118450,26.23267,0.853117,0.146883,0

In [396]:
# 중복확인
df = pl.read_parquet(r"C:\Users\user\OneDrive\dataset\output\features\features_mingle_stage3.parquet")

# ① 구조적 중복: 동일 행정동×분기 조합이 여러 개인 그룹
dup_len = (
    df.group_by(["행정동_코드","기준_년분기_코드"])
      .len()
      .filter(pl.col("len") > 1)
)

# ② 내용적 완전중복: 주요값이 모두 동일한 행의 개수
subset_cols = ["행정동_코드","기준_년분기_코드","총점포수","총생활인구수","총매출액"]
dup_content = df.height - df.unique(subset=subset_cols).height

print(f"구조적 중복 건수 : {dup_len.height:,}")
print(f"내용적 완전중복 건수 : {dup_content:,}")


구조적 중복 건수 : 10,008
내용적 완전중복 건수 : 0


QC

In [420]:
m3 = pl.read_parquet(r"C:\Users\user\OneDrive\dataset\output\features\features_mingle_stage3.parquet")
out = m3.select([
    pl.min("매출민감성지수").alias("min_매출민감성지수"),
    pl.max("매출민감성지수").alias("max_매출민감성지수"),
])
print({"exists": "매출민감성지수" in m3.columns, "range": out.row(0)})


{'exists': True, 'range': (0.048319999999961345, 0.9699199999992241)}


## [B1] 4차 파생 변수 생성 (탄력성 확장 - 외부요인변화에 대한 반응)
: 생활인구 / 점포수 / 매출을 함께 시차변수(lag) 구조로 구성
* 탄력성회귀식 = ln(Salest​)=α+β1​ln(Popt​)+β2​ln(Storet​)+ϵt​
- 유입탄력성(β₁): 외생 수요 민감도, 생활인구변화가 매출변화로 이어지는 정도
- 공급탄력성(β₂): 공급(경쟁) 민감도, 점포수변화가 매출변화로 이어지는 정
- 총탄력성(β₁+β₂): 상권 총 반응성

In [398]:
import polars as pl, numpy as np
from pathlib import Path
from sklearn.linear_model import LinearRegression

def make_mingle_stage4(pop_path, store_path, sales_path, out_path):
    # 1️⃣ 데이터 로드
    pop   = pl.read_parquet(pop_path)
    store = pl.read_parquet(store_path)
    sales = pl.read_parquet(sales_path)

    # 컬럼명 자동 보정 (행정동코드 → 행정동_코드)
    def normalize_cols(df: pl.DataFrame) -> pl.DataFrame:
        rename_map = {}
        if "행정동코드" in df.columns:
            rename_map["행정동코드"] = "행정동_코드"
        if "기준년분기_코드" in df.columns:
            rename_map["기준년분기_코드"] = "기준_년분기_코드"
        return df.rename(rename_map) if rename_map else df

    pop, store, sales = map(normalize_cols, [pop, store, sales])

    # 필요한 컬럼만 추출
    pop   = pop.select(["행정동_코드","기준_년분기_코드","총생활인구수"])
    store = store.select(["행정동_코드","기준_년분기_코드","총점포수"])
    sales = sales.select(["행정동_코드","기준_년분기_코드","총매출액"])

    # 타입 통일
    for df_name, df in [("pop", pop), ("store", store), ("sales", sales)]:
        if df["행정동_코드"].dtype != pl.Int64:
            df = df.with_columns(pl.col("행정동_코드").cast(pl.Int64))
        if df_name == "pop":
            pop = df
        elif df_name == "store":
            store = df
        else:
            sales = df

    # 병합
    df = (
        sales.join(pop,   on=["행정동_코드","기준_년분기_코드"], how="inner")
             .join(store, on=["행정동_코드","기준_년분기_코드"], how="inner")
             .sort(["행정동_코드","기준_년분기_코드"])
    )

    # 로그 변환
    for c in ["총매출액","총생활인구수","총점포수"]:
        df = df.with_columns((pl.col(c)+1).log().alias(f"log_{c}"))

    # 시차 생성 (t-1)
    df = (
        df.with_columns([
            pl.col("log_총매출액").shift(1).over("행정동_코드").alias("log_총매출액_lag"),
            pl.col("log_총생활인구수").shift(1).over("행정동_코드").alias("log_총생활인구수_lag"),
            pl.col("log_총점포수").shift(1).over("행정동_코드").alias("log_총점포수_lag"),
        ])
        .drop_nulls(subset=["log_총매출액_lag","log_총생활인구수_lag","log_총점포수_lag"])
    )

    # 변화율 (Δln)
    df = df.with_columns([
        (pl.col("log_총매출액") - pl.col("log_총매출액_lag")).alias("dlog_sales"),
        (pl.col("log_총생활인구수") - pl.col("log_총생활인구수_lag")).alias("dlog_pop"),
        (pl.col("log_총점포수") - pl.col("log_총점포수_lag")).alias("dlog_store")
    ])

    # 2️⃣ 회귀: Δln(Sales) = α + β1*Δln(Pop) + β2*Δln(Store)
    results = []
    for code, grp in df.group_by("행정동_코드", maintain_order=True):
        try:
            x = grp.select(["dlog_pop","dlog_store"]).to_numpy()
            y = grp.select("dlog_sales").to_numpy().ravel()
            if len(y) < 3:
                continue
            model = LinearRegression(fit_intercept=True).fit(x, y)
            results.append((code, model.coef_[0], model.coef_[1]))
        except Exception:
            continue

    res_df = pl.DataFrame(results, schema=["행정동_코드","유입탄력성","공급탄력성"])
    res_df = res_df.with_columns((pl.col("유입탄력성")+pl.col("공급탄력성")).alias("총탄력성"))

    # 스케일링 0~1
    for c in ["유입탄력성","공급탄력성","총탄력성"]:
        res_df = res_df.with_columns(
            (((pl.col(c) - pl.col(c).min()) / (pl.col(c).max() - pl.col(c).min())).clip(0,1)).alias(f"{c}_norm")
        )

    # 저장
    Path(out_path).parent.mkdir(parents=True, exist_ok=True)
    res_df.write_parquet(out_path)
    print(f"✅ 4차 탄력성(유입·공급·총) 계산 완료 → {out_path}")
    return res_df


make_mingle_stage4(
    pop_path   = r"C:\Users\user\OneDrive\dataset\output\features\features_pop.parquet",
    store_path = r"C:\Users\user\OneDrive\dataset\output\features\features_store.parquet",
    sales_path = r"C:\Users\user\OneDrive\dataset\output\features\features_sales_clean.parquet",
    out_path   = r"C:\Users\user\OneDrive\dataset\output\features\features_mingle_stage4.parquet"
)


✅ 4차 탄력성(유입·공급·총) 계산 완료 → C:\Users\user\OneDrive\dataset\output\features\features_mingle_stage4.parquet


C:\Users\user\AppData\Local\Temp\ipykernel_6640\168359154.py:79: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  res_df = pl.DataFrame(results, schema=["행정동_코드","유입탄력성","공급탄력성"])


행정동_코드,유입탄력성,공급탄력성,총탄력성,유입탄력성_norm,공급탄력성_norm,총탄력성_norm
list[i64],f64,f64,f64,f64,f64,f64
[11110515],0.457992,1.187351,1.645343,0.675153,0.46356,0.452115
[11110530],0.518621,7.188241,7.706861,0.704728,0.82547,0.802464
[11110540],0.818261,1.494553,2.312814,0.850895,0.482087,0.490694
[11110550],0.437872,0.535263,0.973135,0.665338,0.424233,0.413262
[11110560],0.218543,3.277568,3.496111,0.558349,0.58962,0.559087
…,…,…,…,…,…,…
[11740640],-0.361764,2.670342,2.308578,0.275271,0.552998,0.490449
[11740650],-0.337529,3.412774,3.075245,0.287093,0.597774,0.534761
[11740660],0.208063,0.655524,0.863587,0.553236,0.431486,0.40693


In [402]:
# 중복확인
df = pl.read_parquet(r"C:\Users\user\OneDrive\dataset\output\features\features_mingle_stage4.parquet")

# ① 구조적 중복: 동일 행정동×분기 조합이 여러 개인 그룹
dup_len = (
    df.group_by(["행정동_코드"])
      .len()
      .filter(pl.col("len") > 1)
)

# ② 내용적 완전중복: 주요값이 모두 동일한 행의 개수
subset_cols = ["행정동_코드","유입탄력성","공급탄력성","총탄력성"]
dup_content = df.height - df.unique(subset=subset_cols).height

print(f"구조적 중복 건수 : {dup_len.height:,}")
print(f"내용적 완전중복 건수 : {dup_content:,}")


구조적 중복 건수 : 0
내용적 완전중복 건수 : 0


QC

In [438]:
import polars as pl

df = pl.read_parquet(r"C:\Users\user\OneDrive\dataset\output\features\features_mingle_stage4.parquet")

print("📊 [Stage4 QC 요약]")
print(f"• Shape: {df.shape}")

# 1️⃣ 결측치
na_sum = df.null_count().to_dict()
print(f"• 결측치 총합: {sum(na_sum.values())}")

# 2️⃣ 무한대값
float_cols = [c for c, dt in df.schema.items() if dt in (pl.Float32, pl.Float64)]
has_inf = any(df.select([pl.col(c).is_infinite().any().alias(c) for c in float_cols]).to_series(0))
print(f"• ∞ 존재: {'있음' if has_inf else '없음'}")

# 3️⃣ 정규화 컬럼 범위
rng = df.select(
    pl.min("총탄력성_norm").alias("min_norm"),
    pl.max("총탄력성_norm").alias("max_norm")
).row(0)
print(f"• 총탄력성_norm 범위: {rng}")

print("✅ QC 완료")


📊 [Stage4 QC 요약]
• Shape: (417, 7)
• 결측치 총합: shape: (1,)
Series: '행정동_코드' [u32]
[
	0
]
• ∞ 존재: 없음
• 총탄력성_norm 범위: (0.0, 1.0)
✅ QC 완료


## [B2: 외부 반응 강도]
## 5차 파생 변수 생성
## (탄력성 * pop, 탄력성 * store 등 외부변화율과 탄력성의 결합)
ex) 
- 거주인구증감률, 직장인구증감률, 점포순증율, 매출증감률
- 유입탄력성_norm × 거주인구증감률 → 유입반응지수
- 공급탄력성_norm × 점포순증율 → 공급반응지수
- 탄력성_norm × 매출증감률 → 총반응지수
* 최종 산출: 상권 외부반응성 지수=𝑓(유입반응지수,공급반응지수,총반응지수)

In [446]:
import polars as pl
from pathlib import Path

# ============ 공용 유틸 ============

def _normalize_cols(df: pl.DataFrame) -> pl.DataFrame:
    """행정동코드/기준년분기_코드 → 행정동_코드/기준_년분기_코드 표준화"""
    m = {}
    if "행정동코드" in df.columns:
        m["행정동코드"] = "행정동_코드"
    if "기준년분기_코드" in df.columns:
        m["기준년분기_코드"] = "기준_년분기_코드"
    return df.rename(m) if m else df

def _fix_code_type(df: pl.DataFrame) -> pl.DataFrame:
    """행정동_코드 dtype/list 문제 정리 → Int64 단일 컬럼로 통일"""
    if "행정동_코드" in df.columns:
        dt = df.schema["행정동_코드"]
        if dt == pl.List(pl.Int64) or dt == pl.List(pl.Utf8):
            df = df.with_columns(pl.col("행정동_코드").list.first().alias("행정동_코드"))
            dt = df.schema["행정동_코드"]
        if dt == pl.Utf8:
            df = df.with_columns(pl.col("행정동_코드").cast(pl.Int64))
        elif dt in (pl.Int8, pl.Int16, pl.Int32, pl.UInt8, pl.UInt16, pl.UInt32, pl.UInt64):
            df = df.with_columns(pl.col("행정동_코드").cast(pl.Int64))
    return df

def _ensure_quarter_int(df: pl.DataFrame) -> pl.DataFrame:
    if "기준_년분기_코드" in df.columns and df["기준_년분기_코드"].dtype != pl.Int64:
        df = df.with_columns(pl.col("기준_년분기_코드").cast(pl.Int64))
    return df

def _pct_rank(expr: pl.Expr) -> pl.Expr:
    """분기 내 percent-rank 0~1"""
    EPS = 1e-9
    r = expr.rank(method="average").over("기준_년분기_코드")
    n = pl.count().over("기준_년분기_코드")
    return ((r - 1) / (n - 1 + EPS)).fill_nan(0).fill_null(0).clip(0, 1)

def _dedup_by_keys(df: pl.DataFrame, keys=("행정동_코드","기준_년분기_코드"), how="mean") -> pl.DataFrame:
    """키 중복이 있으면 수치컬럼은 평균, 비수치는 첫값으로 요약"""
    if not set(keys).issubset(df.columns):
        return df
    # 이미 유니크면 그대로 반환
    dups = (df.group_by(list(keys)).len().filter(pl.col("len") > 1)).height
    if dups == 0:
        return df

    num_cols = [c for c, t in df.schema.items() if t.is_numeric() and c not in keys]
    other_cols = [c for c in df.columns if c not in keys and c not in num_cols]

    agg_exprs = []
    if how == "mean":
        agg_exprs += [pl.col(c).mean().alias(c) for c in num_cols]
    else:
        agg_exprs += [pl.col(c).first().alias(c) for c in num_cols]
    agg_exprs += [pl.col(c).first().alias(c) for c in other_cols]

    return df.group_by(list(keys)).agg(agg_exprs)

def _assert_unique(df: pl.DataFrame, keys=("행정동_코드","기준_년분기_코드"), name="DF"):
    d = (df.group_by(list(keys)).len().filter(pl.col("len") > 1)).height
    print(f"• {_fmt_name(name)} 유니크 체크({keys}) → 중복 {d}건")
    if d:
        raise ValueError(f"❌ {name}: 키 {keys} 기준 중복 {d}건")

def _fmt_name(x): return str(x)

# ============ Stage B2(final): 외부반응성 지수 ============
def make_mingle_stage5(
    pop_path: str,
    store_path: str,
    sales_path: str,
    stage4_path: str,   # 유입/공급/총 탄력성(행정동 단일키)
    out_path: str
):
    print("\n[Stage5] === 외부반응성 지수 생성 시작 ===")

    # 1) 로드 & 컬럼 표준화
    pop   = _ensure_quarter_int(_fix_code_type(_normalize_cols(pl.read_parquet(pop_path))))
    store = _ensure_quarter_int(_fix_code_type(_normalize_cols(pl.read_parquet(store_path))))
    sales = _ensure_quarter_int(_fix_code_type(_normalize_cols(pl.read_parquet(sales_path))))
    ela   = _fix_code_type(_normalize_cols(pl.read_parquet(stage4_path)))  # 분기 없음(행정동 단일)

    # 2) 필요 컬럼 슬림화
    pop   = pop.select([c for c in ["행정동_코드","기준_년분기_코드","총생활인구수"] if c in pop.columns])
    store = store.select([c for c in ["행정동_코드","기준_년분기_코드","총점포수"] if c in store.columns])
    # sales: 총매출액 필수, 매출증감률 있으면 사용
    sales_cols = [c for c in ["행정동_코드","기준_년분기_코드","총매출액","매출증감률"] if c in sales.columns]
    sales = sales.select(sales_cols)

    # 3) 조인 키 유니크 보장(사전 핀포인트)
    print(f"• pop shape  : {pop.shape}")
    print(f"• store shape: {store.shape}")
    print(f"• sales shape: {sales.shape}")
    print(f"• ela shape  : {ela.shape}")

    pop   = _dedup_by_keys(pop,   ("행정동_코드","기준_년분기_코드"))
    store = _dedup_by_keys(store, ("행정동_코드","기준_년분기_코드"))
    sales = _dedup_by_keys(sales, ("행정동_코드","기준_년분기_코드"))
    # ela는 행정동 단일키로 유니크 보장
    if "기준_년분기_코드" in ela.columns:
        ela = ela.drop("기준_년분기_코드")
    ela = _dedup_by_keys(ela, ("행정동_코드",), how="first")

    _assert_unique(pop,   ("행정동_코드","기준_년분기_코드"), "POP")
    _assert_unique(store, ("행정동_코드","기준_년분기_코드"), "STORE")
    _assert_unique(sales, ("행정동_코드","기준_년분기_코드"), "SALES")
    _assert_unique(ela,   ("행정동_코드",),                 "ELA(탄력성)")

    # 4) 패널 병합
    panel = (sales
             .join(pop,   on=["행정동_코드","기준_년분기_코드"], how="inner")
             .join(store, on=["행정동_코드","기준_년분기_코드"], how="inner")
            )
    print(f"• panel(after join sales/pop/store) shape: {panel.shape}")
    _assert_unique(panel, ("행정동_코드","기준_년분기_코드"), "PANEL(base)")

    # 5) 로그 + 랙 + Δlog
    panel = panel.sort(["행정동_코드","기준_년분기_코드"]).with_columns([
        (pl.col("총매출액")+1).log().alias("log_sales"),
        (pl.col("총생활인구수")+1).log().alias("log_pop"),
        (pl.col("총점포수")+1).log().alias("log_store"),
    ]).with_columns([
        pl.col("log_sales").shift(1).over("행정동_코드").alias("log_sales_L1"),
        pl.col("log_pop").shift(1).over("행정동_코드").alias("log_pop_L1"),
        pl.col("log_store").shift(1).over("행정동_코드").alias("log_store_L1"),
    ]).with_columns([
        (pl.col("log_sales") - pl.col("log_sales_L1")).fill_null(0).alias("dlog_sales"),
        (pl.col("log_pop")   - pl.col("log_pop_L1")).fill_null(0).alias("dlog_pop"),
        (pl.col("log_store") - pl.col("log_store_L1")).fill_null(0).alias("dlog_store"),
    ])

    if "매출증감률" not in panel.columns:
        panel = panel.with_columns(pl.col("dlog_sales").alias("매출증감률"))

    # 6) 탄력성 조인(브로드캐스트: 행정동 단일키)
    need_ela = {"행정동_코드","유입탄력성_norm","공급탄력성_norm","총탄력성_norm"}
    miss_ela = need_ela - set(ela.columns)
    if miss_ela:
        raise ValueError(f"❌ Stage4(탄력성) 누락 컬럼: {sorted(miss_ela)}")

    # 혹시 모를 타입 이슈 재정리
    ela = _fix_code_type(ela).select(list(need_ela))

    before = panel.shape
    panel = panel.join(ela, on="행정동_코드", how="left")
    print(f"• panel(after join ELA) shape: {before} → {panel.shape}")
    # 여기서 행 증폭이 보이면 ela에 중복이 있는 것 → 바로 예외
    _assert_unique(ela, ("행정동_코드",), "ELA(recheck)")
    _assert_unique(panel, ("행정동_코드","기준_년분기_코드"), "PANEL(+ELA)")

    # 7) 변화강도(분기 내 상대화, 절대값)
    panel = panel.with_columns([
        _pct_rank(pl.col("dlog_pop").abs()).alias("변화강도_pop"),
        _pct_rank(pl.col("dlog_store").abs()).alias("변화강도_store"),
        _pct_rank(pl.col("dlog_sales").abs()).alias("변화강도_sales"),
    ])

    # 8) 반응지수(탄력성 × 변화강도)
    panel = panel.with_columns([
        (pl.col("유입탄력성_norm") * pl.col("변화강도_pop")).clip(0,1).alias("유입반응지수"),
        (pl.col("공급탄력성_norm") * pl.col("변화강도_store")).clip(0,1).alias("공급반응지수"),
        (pl.col("총탄력성_norm")   * pl.col("변화강도_sales")).clip(0,1).alias("총반응지수"),
    ])

    # 9) 외부반응성 지수 및 점수  ✅ 두 번에 나눠서 생성
    panel = panel.with_columns(
        (0.4*pl.col("유입반응지수") + 0.4*pl.col("공급반응지수") + 0.2*pl.col("총반응지수"))
        .alias("상권외부반응성지수")
    )
    
    panel = panel.with_columns([
        (pl.col("유입반응지수")*100).round(3).alias("유입반응지수_100"),
        (pl.col("공급반응지수")*100).round(3).alias("공급반응지수_100"),
        (pl.col("총반응지수")*100).round(3).alias("총반응지수_100"),
        (pl.col("상권외부반응성지수")*100).round(3).alias("상권외부반응성지수_100"),
    ])


    # 10) 저장(키/핵심 컬럼만 저장)
    Path(out_path).parent.mkdir(parents=True, exist_ok=True)
    out_cols = [
        "행정동_코드","기준_년분기_코드",
        "유입탄력성_norm","공급탄력성_norm","총탄력성_norm",
        "dlog_pop","dlog_store","dlog_sales",
        "변화강도_pop","변화강도_store","변화강도_sales",
        "유입반응지수","공급반응지수","총반응지수","상권외부반응성지수",
        "유입반응지수_100","공급반응지수_100","총반응지수_100","상권외부반응성지수_100"
    ]
    panel.select(out_cols).write_parquet(out_path)
    print(f"✅ Stage5 완료 → {out_path}")
    return panel


make_mingle_stage5(
    pop_path   = r"C:\Users\user\OneDrive\dataset\output\features\features_pop.parquet",
    store_path = r"C:\Users\user\OneDrive\dataset\output\features\features_store.parquet",
    sales_path = r"C:\Users\user\OneDrive\dataset\output\features\features_sales_clean.parquet",
    stage4_path= r"C:\Users\user\OneDrive\dataset\output\features\features_mingle_stage4.parquet",
    out_path   = r"C:\Users\user\OneDrive\dataset\output\features\features_mingle_stage5.parquet"
)



[Stage5] === 외부반응성 지수 생성 시작 ===
• pop shape  : (30528, 3)
• store shape: (10200, 3)
• sales shape: (10168, 4)
• ela shape  : (417, 7)
• POP 유니크 체크(('행정동_코드', '기준_년분기_코드')) → 중복 0건
• STORE 유니크 체크(('행정동_코드', '기준_년분기_코드')) → 중복 0건
• SALES 유니크 체크(('행정동_코드', '기준_년분기_코드')) → 중복 0건
• ELA(탄력성) 유니크 체크(('행정동_코드',)) → 중복 0건
• panel(after join sales/pop/store) shape: (10008, 6)
• PANEL(base) 유니크 체크(('행정동_코드', '기준_년분기_코드')) → 중복 0건
• panel(after join ELA) shape: (10008, 15) → (10008, 18)
• ELA(recheck) 유니크 체크(('행정동_코드',)) → 중복 0건
• PANEL(+ELA) 유니크 체크(('행정동_코드', '기준_년분기_코드')) → 중복 0건
✅ Stage5 완료 → C:\Users\user\OneDrive\dataset\output\features\features_mingle_stage5.parquet


C:\Users\user\AppData\Local\Temp\ipykernel_6640\3556584390.py:37: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  n = pl.count().over("기준_년분기_코드")


행정동_코드,기준_년분기_코드,총매출액,매출증감률,총생활인구수,총점포수,log_sales,log_pop,log_store,log_sales_L1,log_pop_L1,log_store_L1,dlog_sales,dlog_pop,dlog_store,공급탄력성_norm,유입탄력성_norm,총탄력성_norm,변화강도_pop,변화강도_store,변화강도_sales,유입반응지수,공급반응지수,총반응지수,상권외부반응성지수,유입반응지수_100,공급반응지수_100,총반응지수_100,상권외부반응성지수_100
i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
11110515,20191,15568061933,0.0,95799.835165,753.0,23.468487,11.470027,6.625392,null,null,null,0.0,0.0,0.0,0.46356,0.675153,0.452115,0.5,0.5,0.5,0.337577,0.23178,0.226057,0.272954,33.758,23.178,22.606,27.295
11110515,20192,16874853653,0.083941,98130.461235,769.0,23.54909,11.494063,6.646391,23.468487,11.470027,6.625392,0.080603,0.024037,0.020998,0.46356,0.675153,0.452115,0.262019,0.867788,0.622596,0.176903,0.402272,0.281485,0.287967,17.69,40.227,28.148,28.797
11110515,20193,16424404096,-0.026694,100076.935554,778.0,23.522034,11.513705,6.658011,23.54909,11.494063,6.646391,-0.027056,0.019641,0.011621,0.46356,0.675153,0.452115,0.334135,0.649038,0.442308,0.225592,0.300868,0.199974,0.250579,22.559,30.087,19.997,25.058
11110515,20194,17702770570,0.077833,105180.061835,777.0,23.596987,11.563439,6.656727,23.522034,11.513705,6.658011,0.074953,0.049734,-0.001285,0.46356,0.675153,0.452115,0.685096,0.03125,0.591346,0.462545,0.014486,0.267356,0.244284,46.254,1.449,26.736,24.428
11110515,20201,15611509280,-0.118132,93331.141031,781.0,23.471274,11.44392,6.661855,23.596987,11.563439,6.656727,-0.125713,-0.119519,0.005128,0.46356,0.675153,0.452115,0.923077,0.338942,0.697115,0.623218,0.15712,0.315176,0.375171,62.322,15.712,31.518,37.517
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
11740700,20234,27657176101,-0.078556,164805.379653,1090.0,24.043151,12.012527,6.99485,24.124965,12.008207,6.993015,-0.081814,0.00432,0.001835,0.488055,0.133729,0.411438,0.15625,0.170673,0.822115,0.020895,0.083298,0.338249,0.109327,2.09,8.33,33.825,10.933
11740700,20241,27507243448,-0.005421,166550.651599,1103.0,24.037715,12.023061,7.006695,24.043151,12.012527,6.99485,-0.005436,0.010534,0.011845,0.488055,0.133729,0.411438,0.28125,0.762019,0.168269,0.037611,0.371907,0.069232,0.177654,3.761,37.191,6.923,17.765
11740700,20242,27029802284,-0.017357,157766.111576,1093.0,24.020206,11.968875,6.997596,24.037715,12.023061,7.006695,-0.017509,-0.054186,-0.009099,0.488055,0.133729,0.411438,0.776442,0.600962,0.274038,0.103833,0.293302,0.11275,0.181404,10.383,29.33,11.275,18.14


In [454]:
# 중복확인
df = pl.read_parquet(r"C:\Users\user\OneDrive\dataset\output\features\features_mingle_stage5.parquet")

# ① 구조적 중복: 동일 행정동×분기 조합이 여러 개인 그룹
dup_len = (
    df.group_by(["행정동_코드","기준_년분기_코드"])
      .len()
      .filter(pl.col("len") > 1)
)

# ② 내용적 완전중복: 주요값이 모두 동일한 행의 개수
#subset_cols = ["행정동_코드","기준_년분기_코드","변화강도_pop","변화강도_store","변화강도_sales","유입반응지수_100","공급반응지수_100","총반응지수_100","상권외부반응성지수_100"]
subset_cols = ["행정동_코드","기준_년분기_코드"]
dup_content = df.height - df.unique(subset=subset_cols).height

print(f"구조적 중복 건수 : {dup_len.height:,}")
print(f"내용적 완전중복 건수 : {dup_content:,}")


구조적 중복 건수 : 0
내용적 완전중복 건수 : 0


QC

In [458]:
import polars as pl

m5 = pl.read_parquet(r"C:\Users\user\OneDrive\dataset\output\features\features_mingle_stage5.parquet")

# 1) 구조적 중복
dup = (
    m5.group_by(["행정동_코드","기준_년분기_코드"])
      .len()
      .filter(pl.col("len") > 1)
)

# 2) NA 총합 (이 부분은 기존 방식 유지 가능)
na_sum = int(
    m5.null_count()
      .select(pl.sum_horizontal(pl.all()).alias("NA_TOTAL"))
      .item()
)

# 3) ∞(inf) 존재 여부 — 열 단위 집계로 1행 결과 만든 뒤 Python any()
float_cols = [c for c, dt in m5.schema.items() if dt in (pl.Float32, pl.Float64)]
has_inf = any(
    m5.select([pl.col(c).is_infinite().any().alias(c) for c in float_cols]).row(0)
)

print(f"중복: {dup.height:,}  |  NA: {na_sum:,}  |  ∞: {'있음' if has_inf else '없음'}")

# (선택) 중복 예시 상위 5건 보고 싶을 때
if dup.height > 0:
    dup_example = (
        m5.group_by(["행정동_코드","기준_년분기_코드"])
          .len()
          .filter(pl.col("len") > 1)
          .head(5)
    )
    print(dup_example)


중복: 0  |  NA: 0  |  ∞: 없음


## 파생 테이블 리체크

In [460]:
import polars as pl
from pathlib import Path

FILES = {
    "POP"   : r"C:\Users\user\OneDrive\dataset\output\features\features_pop.parquet",
    "STORE" : r"C:\Users\user\OneDrive\dataset\output\features\features_store.parquet",
    "SALES" : r"C:\Users\user\OneDrive\dataset\output\features\features_sales_clean.parquet",
    "STAGE2": r"C:\Users\user\OneDrive\dataset\output\features\features_mingle_stage2.parquet",
    "STAGE3": r"C:\Users\user\OneDrive\dataset\output\features\features_mingle_stage3.parquet",
    "STAGE4": r"C:\Users\user\OneDrive\dataset\output\features\features_mingle_stage4.parquet",
    "STAGE5": r"C:\Users\user\OneDrive\dataset\output\features\features_mingle_stage5.parquet",
}

def sample_preview_full(path: str, n: int = 5, seed: int | None = None):
    df = pl.read_parquet(path)
    k = min(n, df.height) if df.height else 0

    print(f"\n=== {Path(path).name} | shape={df.shape} → sample {k} rows ===")
    # 전체 컬럼/충분한 폭으로 보기 (일시 적용)
    with pl.Config(
        tbl_rows = k if k > 0 else 0,   # 표시할 행 수
        tbl_cols = -1,                  # 모든 컬럼 표시
        tbl_width_chars = 10_000,       # 표 전체 가로폭 충분히 확대
        fmt_str_lengths = 1_000,        # 문자열 잘림 방지
    ):
        out = df.sample(n=k, shuffle=True, seed=seed) if k > 0 else df
        print(out)
        # 컬럼명 전체를 별도로도 확인하고 싶을 때:
        print("• columns:", list(df.columns))
        print("• dtypes :", dict(zip(df.columns, [str(t) for t in df.dtypes])))

# 실행
for name, path in FILES.items():
    try:
        sample_preview_full(path, n=5)  # seed=None → 실행할 때마다 다른 표본
    except Exception as e:
        print(f"[{name}] 로드/표본 추출 중 오류: {e}")



=== features_pop.parquet | shape=(30528, 11) → sample 5 rows ===
shape: (5, 11)
┌────────────┬─────────────────┬───────────────┬──────────────┬───────────────┬──────────────┬──────────────┬──────────┬────────────┬──────────────────┬────────────────────────┐
│ 행정동코드 ┆ 기준년분기_코드 ┆ 주간생활인구  ┆ 야간생활인구 ┆ 총생활인구수  ┆ 주간생활비율 ┆ 야간생활비율 ┆ 남녀비율 ┆ 고령화비율 ┆ 생활인구표준편차 ┆ 파일명                 │
│ ---        ┆ ---             ┆ ---           ┆ ---          ┆ ---           ┆ ---          ┆ ---          ┆ ---      ┆ ---        ┆ ---              ┆ ---                    │
│ str        ┆ i32             ┆ f64           ┆ f64          ┆ f64           ┆ f64          ┆ f64          ┆ f64      ┆ f64        ┆ f64              ┆ str                    │
╞════════════╪═════════════════╪═══════════════╪══════════════╪═══════════════╪══════════════╪══════════════╪══════════╪════════════╪══════════════════╪════════════════════════╡
│ 11680720   ┆ 20232           ┆ 96736.27552   ┆ 48695.520014 ┆ 145431.795534 ┆ 0.665166   

STAGE5 (외부반응성): 예 — 키/범위/타입 모두 정상입니다. (417×24=10,008 행 일치)

STAGE4 (탄력성): 부분주의 — 행정동_코드가 list[i64]로 저장됨(조인 시 list.first로 풀어써서 STAGE5는 정상). 파일 자체를 scalar로 리세이브 권고.

STAGE3/2: 아니오(중복 원인 존재) — **STAGE2 = 30,024 (= 10,008×3)**인 이유는 STAGE1 POP이 분기당 3개(월별 잔존)라서입니다.

STAGE1 POP: 아니오(집계 미완) — 컬럼명이 행정동코드(언더바 없음)이고, 분기집계가 월별 3건씩 남아 PK(행정동×분기) 유니크가 깨진 상태입니다. 파일명 컬럼에 ..._YYYYMM가 남아 있어요.

## pop repair

In [462]:
import polars as pl
from pathlib import Path

POP_IN  = r"C:\Users\user\OneDrive\dataset\output\features\features_pop.parquet"
POP_OUT = r"C:\Users\user\OneDrive\dataset\output\features\features_pop.parquet"  # 덮어쓰기

def repair_pop_quarter_unique(pop_path: str, out_path: str):
    df = pl.read_parquet(pop_path)

    # 1) 컬럼 표준화
    m = {}
    if "행정동코드" in df.columns:      m["행정동코드"] = "행정동_코드"
    if "기준년분기_코드" in df.columns:  m["기준년분기_코드"] = "기준_년분기_코드"
    df = df.rename(m) if m else df

    # 2) 파일명 등 월 단서를 제거하고, 분기단위로 '평균(또는 합)' 집계
    #   - 주간/야간/총생활인구수/비율/표준편차/남녀비율/고령화비율은 '평균' 추천
    #   - 총생활인구수는 원자료 정의에 따라 '평균'을 권장(월별 스냅샷 평균)
    num_cols = [c for c, t in df.schema.items() if t.is_numeric() and c not in ("행정동_코드","기준_년분기_코드")]
    df_q = (
        df.group_by(["행정동_코드","기준_년분기_코드"])
          .agg([pl.col(c).mean().alias(c) for c in num_cols])
          .sort(["행정동_코드","기준_년분기_코드"])
    )

    # 3) 유일키 검증
    dup = (
        df_q.group_by(["행정동_코드","기준_년분기_코드"])
            .len()
            .filter(pl.col("len")>1)
            .height
    )
    if dup:
        raise ValueError(f"POP 유일키 재검증 실패: 중복 {dup}건")

    # 4) 저장
    Path(out_path).parent.mkdir(parents=True, exist_ok=True)
    df_q.write_parquet(out_path)
    print(f"✅ POP 리페어 완료 → {out_path} | shape={df_q.shape}")

repair_pop_quarter_unique(POP_IN, POP_OUT)


✅ POP 리페어 완료 → C:\Users\user\OneDrive\dataset\output\features\features_pop.parquet | shape=(10176, 10)


## stage 2 / 3 / 5 재실행

In [464]:
# STAGE2
make_mingle_stage2(
    pop_path   = r"C:\Users\user\OneDrive\dataset\output\features\features_pop.parquet",
    store_path = r"C:\Users\user\OneDrive\dataset\output\features\features_store.parquet",
    sales_clean_path = r"C:\Users\user\OneDrive\dataset\output\features\features_sales_clean.parquet",
    out_path  = r"C:\Users\user\OneDrive\dataset\output\features\features_mingle_stage2.parquet"
)

# STAGE3
make_mingle_stage3(
    in_path = r"C:\Users\user\OneDrive\dataset\output\features\features_mingle_stage2.parquet",
    out_path= r"C:\Users\user\OneDrive\dataset\output\features\features_mingle_stage3.parquet"
)

# STAGE5 (STAGE4는 그대로 사용 가능)
make_mingle_stage5(
    pop_path   = r"C:\Users\user\OneDrive\dataset\output\features\features_pop.parquet",
    store_path = r"C:\Users\user\OneDrive\dataset\output\features\features_store.parquet",
    sales_path = r"C:\Users\user\OneDrive\dataset\output\features\features_sales_clean.parquet",
    stage4_path= r"C:\Users\user\OneDrive\dataset\output\features\features_mingle_stage4.parquet",
    out_path   = r"C:\Users\user\OneDrive\dataset\output\features\features_mingle_stage5.parquet"
)

# STAGE4는 list를 scalar로 리세브브
in4  = r"C:\Users\user\OneDrive\dataset\output\features\features_mingle_stage4.parquet"
out4 = in4  # 덮어쓰기

df4 = pl.read_parquet(in4)
if isinstance(df4.schema["행정동_코드"], pl.datatypes.List):
    df4 = df4.with_columns(pl.col("행정동_코드").list.first().alias("행정동_코드"))
df4.write_parquet(out4)
print("✅ STAGE4 리세이브(list→scalar) 완료")


✅ 2차 밍글링 완료 → C:\Users\user\OneDrive\dataset\output\features\features_mingle_stage2.parquet
✅ 3차 지수화 완료 → C:\Users\user\OneDrive\dataset\output\features\features_mingle_stage3.parquet

[Stage5] === 외부반응성 지수 생성 시작 ===
• pop shape  : (10176, 3)
• store shape: (10200, 3)
• sales shape: (10168, 4)
• ela shape  : (417, 7)


C:\Users\user\AppData\Local\Temp\ipykernel_6640\2722364337.py:24: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  n = pl.count().over("기준_년분기_코드")


• POP 유니크 체크(('행정동_코드', '기준_년분기_코드')) → 중복 0건
• STORE 유니크 체크(('행정동_코드', '기준_년분기_코드')) → 중복 0건
• SALES 유니크 체크(('행정동_코드', '기준_년분기_코드')) → 중복 0건
• ELA(탄력성) 유니크 체크(('행정동_코드',)) → 중복 0건
• panel(after join sales/pop/store) shape: (10008, 6)
• PANEL(base) 유니크 체크(('행정동_코드', '기준_년분기_코드')) → 중복 0건
• panel(after join ELA) shape: (10008, 15) → (10008, 18)
• ELA(recheck) 유니크 체크(('행정동_코드',)) → 중복 0건
• PANEL(+ELA) 유니크 체크(('행정동_코드', '기준_년분기_코드')) → 중복 0건
✅ Stage5 완료 → C:\Users\user\OneDrive\dataset\output\features\features_mingle_stage5.parquet
✅ STAGE4 리세이브(list→scalar) 완료


C:\Users\user\AppData\Local\Temp\ipykernel_6640\3556584390.py:37: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  n = pl.count().over("기준_년분기_코드")


## 데이터 재재확인 (10times random sampling)

In [478]:
import polars as pl
import random

FILES = {
    "POP"   : r"C:\Users\user\OneDrive\dataset\output\features\features_pop.parquet",
    "STORE" : r"C:\Users\user\OneDrive\dataset\output\features\features_store.parquet",
    "SALES" : r"C:\Users\user\OneDrive\dataset\output\features\features_sales_clean.parquet",
    "STAGE2": r"C:\Users\user\OneDrive\dataset\output\features\features_mingle_stage2.parquet",
    "STAGE3": r"C:\Users\user\OneDrive\dataset\output\features\features_mingle_stage3.parquet",
    "STAGE5": r"C:\Users\user\OneDrive\dataset\output\features\features_mingle_stage5.parquet",
}

# 존재하는 키 자동 선택(POP의 '행정동코드/기준년분기_코드' 등 혼용 대응)
CAND_KEYS = [
    ["행정동_코드", "행정동코드"],
    ["기준_년분기_코드", "기준년분기_코드"],
]
def pick_keys(df: pl.DataFrame):
    keys = []
    for group in CAND_KEYS:
        for k in group:
            if k in df.columns:
                keys.append(k); break
    return keys if len(keys)==2 else None

def basic_qc(df: pl.DataFrame, key_cols: list[str] | None):
    # 1) 구조적 중복
    dup = 0
    if key_cols:
        dup = int(df.group_by(key_cols).len().filter(pl.col("len") > 1).height)

    # 2) 결측치 총합 (Series 합산 문제 방지)
    na_counts = df.null_count()  # 1×N
    na_sum = int(
        na_counts.select(pl.sum_horizontal(pl.all()).alias("NA_TOTAL")).to_series(0)[0]
    )

    # 3) ∞(inf) 존재 여부 — float만 검사
    float_cols = [c for c, dt in df.schema.items() if dt in (pl.Float32, pl.Float64)]
    has_inf = bool(
        df.select(pl.any_horizontal([pl.col(c).is_infinite() for c in float_cols]).alias("HAS_INF"))
          .to_series(0)[0]
    ) if float_cols else False

    return dup, na_sum, has_inf

def sample_and_qc_once(seed=None):
    random.seed(seed)
    print(f"\n=== ROUND seed={seed} ===")
    for name, path in FILES.items():
        df = pl.read_parquet(path)
        keys = pick_keys(df)

        dup, na_sum, has_inf = basic_qc(df, keys)
        print(f"[{name}] shape={df.shape}  |  dup={dup:,}  na={na_sum:,}  inf={'Y' if has_inf else 'N'}")

        k = min(5, df.height)
        samp = df.sample(k, shuffle=True, seed=random.randint(0, 10_000)) if k > 0 else df
        with pl.Config(tbl_rows=k, tbl_cols=-1, tbl_width_chars=10_000, fmt_str_lengths=1_000):
            print(samp)

# 10회 반복
for r in range(1, 11):
    sample_and_qc_once(seed=10_000 + r)



=== ROUND seed=10001 ===
[POP] shape=(10176, 10)  |  dup=0  na=0  inf=N
shape: (5, 10)
┌─────────────┬──────────────────┬───────────────┬──────────────┬───────────────┬──────────────┬──────────────┬──────────┬────────────┬──────────────────┐
│ 행정동_코드 ┆ 기준_년분기_코드 ┆ 주간생활인구  ┆ 야간생활인구 ┆ 총생활인구수  ┆ 주간생활비율 ┆ 야간생활비율 ┆ 남녀비율 ┆ 고령화비율 ┆ 생활인구표준편차 │
│ ---         ┆ ---              ┆ ---           ┆ ---          ┆ ---           ┆ ---          ┆ ---          ┆ ---      ┆ ---        ┆ ---              │
│ str         ┆ i32              ┆ f64           ┆ f64          ┆ f64           ┆ f64          ┆ f64          ┆ f64      ┆ f64        ┆ f64              │
╞═════════════╪══════════════════╪═══════════════╪══════════════╪═══════════════╪══════════════╪══════════════╪══════════╪════════════╪══════════════════╡
│ 11680565    ┆ 20244            ┆ 172920.252858 ┆ 60877.061615 ┆ 233797.314473 ┆ 0.739595     ┆ 0.260405     ┆ 0.882843 ┆ 0.283558   ┆ 9502.181547      │
│ 11110690    ┆ 20213            ┆ 18132.

In [1]:
import polars as pl
from pathlib import Path

# 1. 최종 데이터 로드
FEAT_PATH = r"C:\Users\user\OneDrive\dataset\output\features\features_mingle_spatial.parquet"
df = pl.read_parquet(FEAT_PATH)

# 2. 기초 통계량 산출 (Pandas의 describe와 유사)
summary_df = df.to_pandas().describe()

# 3. 저장
OUT_DIR = Path(r"C:\Users\user\OneDrive\dataset\output\_figures")
OUT_DIR.mkdir(parents=True, exist_ok=True)
summary_df.to_csv(OUT_DIR / "data_summary_table.csv", encoding='utf-8-sig')

print(f"✅ 데이터 요약 테이블 저장 완료: {OUT_DIR / 'data_summary_table.csv'}")

✅ 데이터 요약 테이블 저장 완료: C:\Users\user\OneDrive\dataset\output\_figures\data_summary_table.csv
